In [3]:
################## DEFINIZIONI FUNZIONI E VARIABILI UTILI ###################
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from scipy.optimize import curve_fit
import os
import re
from os.path import exists, isfile, join
import subprocess
import time
import urllib.request
import urllib.parse

####
path_to_linemake = f"/home/starcat/LINEE"
path_to_ARES = f"/home/starcat/Modelli/ARES-master"
path_to_abundance = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c/"
path_to_abundance_0 = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c_0/"
path_to_abundance_1 = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c_1/"
path_to_abundance_2 = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c_2/"
path_to_mainfolder = f"/home/starcat/Scrivania/ANALISIATM/"
path_to_atmosphere = f################## DEFINIZIONI FUNZIONI E VARIABILI UTILI ###################
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from scipy.optimize import curve_fit
import os
import re
from os.path import exists, isfile, join
import subprocess
import time
import urllib.request
import urllib.parse

####
path_to_linemake = f"/home/starcat/LINEE"
path_to_ARES = f"/home/starcat/Modelli/ARES-master"
path_to_abundance = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c/"
path_to_abundance_0 = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c_0/"
path_to_abundance_1 = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c_1/"
path_to_abundance_2 = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c_2/"
path_to_mainfolder = f"/home/starcat/Scrivania/ANALISIATM/"
path_to_atmosphere = f"/home/starcat/Scrivania/ANALISIATM/ATMOSFERE/"
path_to_rnd = f"/home/starcat/Scrivania/ANALISIATM/rnd"
path_to_output = f"/home/starcat/Scrivania/ANALISIATM/ABBONDANZE_OUT/"
path_to_interpol = f"/home/starcat/Scrivania/ANALISIATM/interpol/PyKMOD-main/"
path_to_correctionGrids = f"/home/starcat/Scrivania/ANALISIATM/CorrectionGRID/"
####

N_stelle = 14
trials = 30
temp_log = np.zeros(trials)
logg_log = np.zeros(trials)
T_slope = np.zeros((trials, 2))
Logg_slope = np.zeros((trials, 2))
temp_best_guess1 = 0
temp_best_guess2 = 0
temp_best_guess3 = 0
temperatura = [3500, 3750, 4000, 4250, 4500, 4750, 5000, 5250, 5500, 5750, 6000, 6250, 6500, 6750, 7000, 7250, 7500, 7750, 8000, 8250, 8500, 8750, 9000, 9250, 9500, 9750, 10000, 10250, 10500, 10750, 11000, 11250, 11500, 11750, 12000, 12250, 12500, 12750, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000, 26000, 27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000, 50000]
logg_arr = [0, 0.50000, 1.00000, 1.50000, 2.00000, 2.50000, 3.00000, 3.50000, 4.00000, 4.50000, 5.00000]
MH = ["05", "02", "00", "-05", "-10", "-15", "-20"]
metallicity = ["0.5", "0.2", "0.0", "-0.5", "-1.0", "-1.5", "-2.0", "-2.5"]
metallic = [0.5, 0.2, 0.0, -0.5, -1.0, -1.5, -2.0, -2.5]
metallic_header = [-2.5, 2.0, -1.5, -1.0, -0.5, 0.0, 0.2, 0.5]
T_EFF_int = np.arange(3500, 5500 + 1, 25)
LOGG_int = np.arange(0.5, 5 + 0.06, 0.05)
MH_int = np.arange(0.0, -2.5 - 0.06, -0.05)
T_EFF_Nint = np.arange(3500, 6000 , 250)
LOGG_Nint = np.arange(0.5, 5 + 0.06, 0.5)
MH_Nint = np.arange(0.0, -2.5 - 0.06, -0.5)

ABUNDANCE_SCALE = [3.16228, 1.58489, 1.00000, 0.31620, 0.10000, 0.03162, 0.01000, 0.00316]
ABUNDANCE_SCALE_header = [0.00316, 0.01000, 0.03162, 0.10000, 0.31620, 1.00000, 1.58489, 3.16228]
minceData = [["HARPN1", 4746, 1.89, -0.56],
             ["HARPN2", 4610, 1.73, -0.58],
             ["HARPN3", 4458, 1.54, -0.56],
             ["HARPN4", 4045, 1.21, -0.64],
             ["HARPN5", 4444, 4,-0.5],
             ["HARPN6", 4722, 1.89, 0.02],
             ["HARPN7", 4725, 2.41, -0.19],
             ["HARPN8", 4444, 4, -0.5],
             ["HARPN9", 4458, 1.54, -0.56],
             ["HARPN10", 4393, 1.08, -1.99],
             ["HARPN11", 4193, 0.71, -1.95],
             ["HARPN12", 4167, 0.82, -1.45],
             ["HARPN13", 4154, 0.52, -2.09],
             ["HARPN14", 4287, 0.70, -1.71],
             ["HARPN15", 4316, 0.87, -1.46],
             ["HARPN16", 4276, 0.70, -2.27],
             ["HARPN17", 4249, 0.79, -1.61]]
NLTE_wavelenght = [3440.610, 
                   436.886,
            3440.990,
            3465.860,
            3475.440,
            3490.570,
            3565.380,
            3570.100,
            3581.190,
            3608.860,
            3618.770,
            3719.920,
            3737.120,
            3727.620,
            3745.560,
            3748.260,
            3758.230,
            3763.790,
            3815.840,
            3820.430,
            3824.440,
            3825.870,
            3840.440,
            3856.370,
            3859.910,
            3922.910,
            4045.810,
            4063.590,
            4071.740,
            4235.940,
            4250.120,
            4250.790,
            4415.120,
            4445.470,
            4494.560,
            4920.500,
            4994.130,
            5044.210,
            5198.700,
            5216.270,
            5225.520,
            5232.940,
            5236.200,
            5242.490,
            5247.040,
            5250.210,
            5269.540,
            5281.790,
            5379.570,
            5383.370,
            5434.521,
            5491.830,
            5586.750,
            5600.220,
            5661.350,
            5662.520,
            5696.090,
            5701.540,
            5705.460,
            5778.450,
            5784.660,
            5855.080,
            5916.250,
            5956.690,
            6065.479,
            6082.710,
            6151.620,
            6173.330,
            6200.310,
            6219.280,
            6240.650,
            6252.560,
            6265.130,
            6297.790,
            6311.500,
            6430.850,
            6498.940,
            6518.370,
            6574.229,
            6593.870,
            6609.110,
            6699.140,
            6726.670,
            6739.521,
            6750.150,
            6793.260,
            6810.260,
            6837.010,
            6854.820,
            6945.200,
            6978.850,
            7401.680,
            7912.870,
            8293.500,
            4233.170,
            4491.390,
            4508.290,
            4576.330,
            4582.840,
            4583.830,
            4620.520,
            4923.930,
            5018.440,
            5169.020,
            5197.580,
            5234.620,
            5264.810,
            5284.110,
            5325.550,
            5414.070,
            5425.260,
            6239.950,
            6247.560,
            6369.460,
            6432.680,
            6456.380,
            6516.080,
            7222.390,
            7224.490,
            7515.830,
            7711.720]

def stampaOra():
    current_time = time.localtime()
    formatted_time = time.strftime("%H:%M:%S", current_time)
    print("L'ora corrente è:", formatted_time)
def correzione(M):
    #return -0.15-0.075*M
    return 0
def findBestAtmosphereinFolder(path, mode, attiva_tolleranza=True, tolleranzaG=0.2, tolleranzaT=0.2, elemento1=26.0, elemento2=26.1, requiredT=0, requiredG=0, excludeT=0, excludeG=0): #tolleranza permette di tener di conto di un'eventuale discrepanza tra l'abbondanza di FeI e FeII (una temperatura può essere più accurata anche se la differenza tra abbondanza di FeI e FeII è maggiore di quella ottenuta per un altro logg e temperatura). Non attivare se si ha solo 1 elemento
    """ output:
     mode T: elem1, slope, std, temp, logg, elem2
     mode G: FeI, FeII, G, M
     mode A: FeI FeII T G M s(FeI) stdFeI stdFeII"""
    old_dir = os.getcwd()
    os.chdir(path)
    a = os.listdir()
    b = []
    os.chdir(old_dir)
    if mode=="T":
        if requiredG==0:
            returner = np.zeros(6)
            for i in range(len(a)):
                if a[i][0]=='T':
                    b.append(a[i])
            H = np.zeros((len(b), 7)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
            min = 10
            j=0
            for i in range(len(b)):
                tempS = simpleAnalysisSingle(path, b[i], 26.0)
                tempD = simpleAnalysisDouble(path, b[i], elemento1, elemento2)
                H[i][0] = tempS[1]
                H[i][1] = float(tempD[0])
                H[i][2] = float(tempD[1])
                H[i][3] = (parseFileName(path, b[i]))[1]
                H[i][4] = (parseFileName(path, b[i]))[2]
                H[i][5] = (parseFileName(path, b[i]))[0]
                H[i][6] = tempS[2]
                if attiva_tolleranza:
                    if (abs(H[i][2]-H[i][1]) < tolleranzaG) and (abs(H[i][0]) < abs(min)):
                        min = H[i][0]
                        j=i
                else:
                    if abs(H[i][2]-H[i][1]) < tolleranzaG and (abs(H[i][0]) < abs(min)):
                        min = H[i][0]
                        j=i
            returner[0]=H[j][1] #el1
            returner[1]=H[j][0] #slope
            returner[2]=H[j][6] #std
            returner[3]=H[j][3] #temp
            returner[4]=H[j][4] #logg
            returner[5]=H[j][2] #el2
            

            return returner
        else:
            returner = np.zeros(6)
            for i in range(len(a)):
                if a[i][0]=='T':
                    b.append(a[i])
            H = np.zeros((len(b), 7)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
            min = 10
            j=0
            for i in range(len(b)):
                tempS = simpleAnalysisSingle(path, b[i], 26.0)
                tempD = simpleAnalysisDouble(path, b[i], elemento1, elemento2)
                H[i][0] = tempS[1]
                H[i][1] = float(tempD[0])
                H[i][2] = float(tempD[1])
                H[i][3] = (parseFileName(path, b[i]))[1]
                H[i][4] = (parseFileName(path, b[i]))[2]
                H[i][5] = (parseFileName(path, b[i]))[0]
                H[i][6] = tempS[2]
                if attiva_tolleranza:
                    if abs(H[i][2]-H[i][1]) < tolleranzaG and (abs(H[i][0]) < abs(min)) and H[i][4]==requiredG:
                        min = H[i][0]
                        j=i
                else:
                    if (abs(H[i][0]) < abs(min)):
                        min = H[i][0]
                        j=i
            returner[0]=H[j][1] #el1
            returner[1]=H[j][0] #slope
            returner[2]=H[j][6] #std
            returner[3]=H[j][3] #temp
            returner[4]=H[j][4] #logg
            returner[5]=H[j][2] #el2
            

            return returner
        
    elif mode=="G":
        returner = np.zeros(4)
        for i in range(len(a)):
            if a[i][0]=='G':
                b.append(a[i])
            H = np.zeros((len(b), 7)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
        min = 10
        j=0
        for i in range(len(b)):
            tempS = simpleAnalysisSingle(path, b[i], 26.0)
            tempD = simpleAnalysisDouble(path, b[i], elemento1, elemento2)
            H[i][0] = tempS[1]
            H[i][1] = float(tempD[0])
            H[i][2] = float(tempD[1])
            H[i][3] = (parseFileName(path, b[i]))[1]
            H[i][4] = (parseFileName(path, b[i]))[2]
            H[i][5] = (parseFileName(path, b[i]))[0]
            H[i][6] = tempS[2]
            if attiva_tolleranza:
                if abs(H[i][2]-H[i][1]) < tolleranzaT and (abs(H[i][0]) < abs(min)):
                    min = H[i][0]
                    j=i
            else:
                if (abs(H[i][0]) < abs(min)):
                    min = H[i][0]
                    j=i
        returner[0]=H[j][1]
        returner[1]=H[j][2]
        returner[2]=H[j][4]
        returner[3]=H[j][5]
        return returner
    elif mode=="A":
        returner = np.zeros(8)
        for i in range(len(a)): #and (a[i][0]!='l' and a[i][0] != 'e'):
            b.append(a[i])
        H = np.zeros((len(b), 8)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
        min = 10
        j=0
        for i in range(len(b)):
            tempD = simpleAnalysisSingle(path, b[i], 26.1)
            tempS = simpleAnalysisSingle(path, b[i], 26.0)
            H[i][0] = tempS[1]
            H[i][1] = float(tempS[0])
            H[i][2] = float(tempD[0])
            H[i][3] = (sparseFileName(path, b[i]))[1]
            H[i][4] = (sparseFileName(path, b[i]))[2]
            H[i][5] = (sparseFileName(path, b[i]))[0]
            H[i][6] = tempS[2]
            H[i][7] = tempD[2]
            if attiva_tolleranza:
                if abs(H[i][2]-H[i][1]) < tolleranzaG and abs(H[i][0]) < tolleranzaT and abs(H[i][5]-H[i][1]) < 0.6 and H[i][6] < 0.2 and H[i][0] < min:
                    min = H[i][0]
                    j=i
            # else:
            #     if (abs(H[i][0]) < abs(min)):
            #         min = H[i][0]
            #         j=i
        returner[0]=H[j][1] 
        returner[1]=H[j][2] 
        returner[2]=H[j][3]
        returner[3]=H[j][4]
        returner[4]=H[j][5]
        returner[5]=H[j][0]
        returner[6]=H[j][6]
        returner[7]=H[j][7]
        return returner
    else:
        print("mode deve essere 'T' oppure 'G', per valutare le rispettivamente le abbondanze dedotte muovendosi su temperatura o logg")
        return None
def parseFileName(path, nome_file):
    olddir=os.getcwd()
    os.chdir(path)
    pattern = f"{nome_file[0]}"+r'STELLA(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    if match:
        returner.append(float(match.group(1)))
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        return returner
    else:
        return None
def sparseFileName(path, nome_file):
    olddir=os.getcwd()
    os.chdir(path)
    pattern = r'STELLA(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    if match:
        returner.append(float(match.group(1)))
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        return returner
    else:
        return None
def sparseFileName2(path, nome_file):
    olddir=os.getcwd()
    os.chdir(path)
    pattern = r'AH_STELLA(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    if match:
        returner.append(float(match.group(1)))
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        return returner
    else:
        return None
def make_set(T, G, M, DT=500, DG=1, DM=1):
    set1 = [[T-DT, G+DG, M-DM], [T, G+DG, M-DM], [T+DT, G+DG, M-DM], [T-DT, G, M-DM]]
    set2 = [[T, G, M-DM], [T+DT, G, M-DM], [T-DT, G-DG, M-DM], [T, G-DG, M-DM]]
    set3 = [[T+DT, G-DG, M-DM],[T-DT, G+DG, M], [T, G+DG, M], [T+DT, G+DG, M]]
    set4 = [[T-DT, G, M], [T, G, M], [T+DT, G, M], [T-DT, G-DG, M]]
    set5 = [[T, G-DG, M], [T+DT, G-DG, M], [T-DT, G+DG, M+DM], [T, G+DG, M+DM]]
    set6 = [[T+DT, G+DG, M+DM], [T-DT, G, M+DM], [T, G, M+DM], [T+DT, G, M+DM]]
    set7 = [[T-DT, G-DG, M+DM], [T, G-DG, M+DM], [T+DT, G-DG, M+DM]]
    SET_ALL = [set1, set2, set3, set4, set5, set6, set7]
    return SET_ALL
def searchPatch(nome_stella, T=4750, G=1.50, M=-1.00, DT=500, DG=1, DM=1):
    SET_ALL = make_set(T-250, G, M)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T-250, G+0.5, M)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T-250, G, M-0.5)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T-250, G+0.5, M-0.5)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T, G+0.5, M)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T, G, M-0.5)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T, G+0.5, M-0.5)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
def findAbundancePatch(atmoParameters, nome_stella, n_istanze=4, path_to_output=path_to_output, label='', multiple_elements=0,  elemento1=26.0, elemento2=26.1):
    """Restituisce: multiple element==0 media, slope, std, multiple element==1 media el1 e media el2"""
    preparaAbundance(nome_stella)
    bashcmd = f"mkdir {path_to_output}{nome_stella}"
    os.system(bashcmd)
    copia("stellaFes", path_to_abundance, path_to_abundance_0, "stellaFes")
    copia("stellaFes", path_to_abundance, path_to_abundance_1, "stellaFes")
    copia("stellaFes", path_to_abundance, path_to_abundance_2, "stellaFes")
    if n_istanze == 1:
        copia(f"atmoT{int(atmoParameters[0][0])}{atmoParameters[0][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[0][2]}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[0][0], atmoParameters[0][1], atmoParameters[0][2]))  #fai paramfile di abundance
        SlanciaAbundance(n_istanze)
        temp = path_to_output
        path_to_output = path_to_output + f"{nome_stella}"
        sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{atmoParameters[0][2]}_{int(atmoParameters[0][0])}_{atmoParameters[0][1]}") #copia l'output di abundance nella cartella degli output
        path_to_output=temp
    if n_istanze == 2:
        copia(f"atmoT{int(atmoParameters[0][0])}{atmoParameters[0][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[0][2]}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[0][0], atmoParameters[0][1], atmoParameters[0][2]), 0)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[1][0])}{atmoParameters[1][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[1][2]}", path_to_abundance_1, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[1][0], atmoParameters[1][1], atmoParameters[1][2]), 1)  #fai paramfile di abundance
        SlanciaAbundance(n_istanze)
        temp = path_to_output
        path_to_output = path_to_output + f"{nome_stella}"
        sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{atmoParameters[0][2]}_{int(atmoParameters[0][0])}_{atmoParameters[0][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS1", path_to_abundance_0, path_to_output, f"{label}STELLA{atmoParameters[1][2]}_{int(atmoParameters[1][0])}_{atmoParameters[1][1]}") #copia l'output di abundance nella cartella degli output
        path_to_output=temp
    if n_istanze == 3:
        copia(f"atmoT{int(atmoParameters[0][0])}{atmoParameters[0][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[0][2]}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[0][0], atmoParameters[0][1], atmoParameters[0][2]), 0)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[1][0])}{atmoParameters[1][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[1][2]}", path_to_abundance_0, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[1][0], atmoParameters[1][1], atmoParameters[1][2]), 1)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[2][0])}{atmoParameters[2][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[2][2]}", path_to_abundance_1, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[2][0], atmoParameters[2][1], atmoParameters[2][2]), 2)  #fai paramfile di abundance
        SlanciaAbundance(n_istanze)
        temp = path_to_output
        path_to_output = path_to_output + f"{nome_stella}"
        sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{atmoParameters[0][2]}_{int(atmoParameters[0][0])}_{atmoParameters[0][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS1", path_to_abundance_0, path_to_output, f"{label}STELLA{atmoParameters[1][2]}_{int(atmoParameters[1][0])}_{atmoParameters[1][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS2", path_to_abundance_1, path_to_output, f"{label}STELLA{atmoParameters[2][2]}_{int(atmoParameters[2][0])}_{atmoParameters[2][1]}") #copia l'output di abundance nella cartella degli output
        path_to_output=temp
    if n_istanze == 4:
        copia(f"atmoT{int(atmoParameters[0][0])}{atmoParameters[0][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[0][2]}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[0][0], atmoParameters[0][1], atmoParameters[0][2]), 0)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[1][0])}{atmoParameters[1][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[1][2]}", path_to_abundance_0, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[1][0], atmoParameters[1][1], atmoParameters[1][2]), 1)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[2][0])}{atmoParameters[2][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[2][2]}", path_to_abundance_1, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[2][0], atmoParameters[2][1], atmoParameters[2][2]), 2)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[3][0])}{atmoParameters[3][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[3][2]}", path_to_abundance_2, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[3][0], atmoParameters[3][1], atmoParameters[3][2]), 3)  #fai paramfile di abundance
        SlanciaAbundance(n_istanze)                                  #lancia abundance
        temp = path_to_output
        path_to_output = path_to_output + f"{nome_stella}"
        sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{atmoParameters[0][2]}_{int(atmoParameters[0][0])}_{atmoParameters[0][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS1", path_to_abundance_0, path_to_output, f"{label}STELLA{atmoParameters[1][2]}_{int(atmoParameters[1][0])}_{atmoParameters[1][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS2", path_to_abundance_1, path_to_output, f"{label}STELLA{atmoParameters[2][2]}_{int(atmoParameters[2][0])}_{atmoParameters[2][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS3", path_to_abundance_2, path_to_output, f"{label}STELLA{atmoParameters[3][2]}_{int(atmoParameters[3][0])}_{atmoParameters[3][1]}") #copia l'output di abundance nella cartella degli output
        path_to_output=temp
    # if multiple_elements == 0:
    #     results = simpleAnalysisSingle(path_to_output, f"TSTELLA{metal}_{int(tempera)}_{logG}", elemento1)
    # else:
    #     results = simpleAnalysisDouble(path_to_output, f"GSTELLA{metal}_{int(tempera)}_{logG}", elemento1, elemento2)
    # return results
def iterative_Search(tempera, logG, metal, trials=8, elemento=26.0, controller=1):
    diarioTemp = np.zeros((trials, 4))
    i=0
    min = 1000
    control = 1
    while controller==1 and i < trials:
        print(f"trial = {i}")
        file_check = f"{path_to_output}/TSTELLA{metal}_{int(tempera)}_{logG}"
        file_checkG = f"{path_to_output}/GSTELLA{metal}_{int(tempera)}_{logG}"
        if (exists(file_check)):
            break
        A = findAbundance(tempera, logG, metal, "T", 0, elemento)
        diarioTemp[i][0] = A[0]
        diarioTemp[i][1] = A[1]
        diarioTemp[i][2] = A[2]
        diarioTemp[i][3] = tempera
        if (i>=1) and (abs(diarioTemp[i][1])>abs(diarioTemp[i-1][1])):
            break
        metal = find_closest(A[0], metallic)
        required_index = 0
        for k in range(trials):
            if (k!=i) and (diarioTemp[k][3]==diarioTemp[i][3]):
                controller = 0
        if ((tempera > 3499) and (tempera < 12750)): 
            if (diarioTemp[i][1]>0):
                tempera = tempera + 250
                tempera = int(tempera)
            else:
                if tempera > 3749:
                    tempera = tempera - 250
                    tempera = int(tempera)
        i=i+1
    if controller == 1:
        print("il numero di trials non è stato sufficiente ad ottenere un risultato, prova con un numero di trials maggiore")
        if i != 0:
            for j in range(trials):
                if (diarioTemp[j][3] != 0) and (abs(diarioTemp[j][1]) < abs(min)): #!= 0 perché viene inizializzato con alcuni 0 che potrebbero non essere utilizzati, in tal caso ci si ferma prima. Si cerca la miglior slope, infatti si ricerca min e via via si sostituisce con la slope più bassa. Alla slope più bassa, la j-esima, corrisponde la j-esima atmosfera
                    min = diarioTemp[j][1]
                    required_index = j
            returner = np.zeros(4)
            returner[0] = diarioTemp[required_index][0] #media
            returner[1] = diarioTemp[required_index][1] #slope
            returner[2] = diarioTemp[required_index][2] #std
            returner[3] = diarioTemp[required_index][3] #temperatura
            return returner
    if controller == 0:
        for j in range(trials):
            if (diarioTemp[j][3] != 0) and (abs(diarioTemp[j][1]) < abs(min)): #!= 0 perché viene inizializzato con alcuni 0 che potrebbero non essere utilizzati, in tal caso ci si ferma prima. Si cerca la miglior slope, infatti si ricerca min e via via si sostituisce con la slope più bassa. Alla slope più bassa, la j-esima, corrisponde la j-esima atmosfera
                min = diarioTemp[j][1]
                required_index = j
        print(f"la temperatura {diarioTemp[required_index][3]} è il valore più vicino a quello cercato \ncorrispondente a una slope di {diarioTemp[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioTemp[required_index][0]} +- {diarioTemp[required_index][2]}")
        returner = np.zeros(4)
        returner[0] = diarioTemp[required_index][0] #media
        returner[1] = diarioTemp[required_index][1] #slope
        returner[2] = diarioTemp[required_index][2] #std
        returner[3] = diarioTemp[required_index][3] #temperatura
        return returner
def linear_func(x, a, b):
    return b +a*x
def expo_func(x, a, b):
    return a * np.exp(b*x)
def crea_ATMinterpol(T, logg, MH_g, nome_Stella, T0=0, logg0=0, MH_g0=0, n_istanze=1): #ritorna il nome del file prodotto da abundance con l'atmosfera creata tramite interpolazione
    '''T: temperatura (kelvin)
    logg: gravità superficiale (dex)
    MH_g: [M/H] (dex)'''
    os.chdir(path_to_interpol)
    if n_istanze == 1:
        vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg

        commandInterpol = f"/home/starcat/anaconda/bin/python pykmod.py {T} {logg} {vturb} {MH_g}"
        os.system(commandInterpol)

        #### FORMATTA KURUCZ ####
        header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
        header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
        tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
        def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
            with open(input_file, 'r') as f_in:
                lines = f_in.readlines()

            # Remove the first 3 rows and the last 2 rows
            lines = lines[3:-2]

            with open(output_file, 'w') as f_out:
                f_out.write(header1)
                f_out.write(header2)
                f_out.writelines(lines)
                f_out.write(tail)

        read_and_copy_file('modelatmosphere.txt', 'out.txt')
        comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
        os.system(comandoATMOS)
        preparaAbundance(nome_Stella)
        os.chdir(path_to_abundance)
        creaParamfileAbnd(vturbo(T, logg, MH_g))
        SlanciaAbundance()
        nome_output = f"{nome_Stella}_INT{MH_g:.2f}_{T:.0f}_{logg:.2f}"
        comando_bsh2 = f"mv PINOLOS {path_to_output}/{nome_output}"
        os.system(comando_bsh2)
        return nome_output
    if n_istanze == 2:
        ##### prima atmosfera
        vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg

        commandInterpol = f"/home/starcat/anaconda/bin/python pykmod.py {T} {logg} {vturb} {MH_g}"
        print(vturb)
        os.system(commandInterpol)

        #### FORMATTA KURUCZ ####
        header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
        header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
        tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
        def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
            with open(input_file, 'r') as f_in:
                lines = f_in.readlines()

            # Remove the first 3 rows and the last 2 rows
            lines = lines[3:-2]

            with open(output_file, 'w') as f_out:
                f_out.write(header1)
                f_out.write(header2)
                f_out.writelines(lines)
                f_out.write(tail)

        read_and_copy_file('modelatmosphere.txt', 'out.txt')
        comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
        os.system(comandoATMOS)
        preparaAbundance(nome_Stella)
        os.chdir(path_to_abundance)
        creaParamfileAbnd(vturbo(T, logg, MH_g))
        ##### seconda atmosfera
        vturb0 = 0.14 - 0.08 * (MH_g0) + 4.9 * T / 10000 - 0.47 * logg0

        commandInterpol = f"/home/starcat/anaconda/bin/python pykmod.py {T0} {logg0} {vturb0} {MH_g0}"
        print(vturb)
        os.system(commandInterpol)

        #### FORMATTA KURUCZ ####
        header1 = f"TEFF   {T0}.  GRAVITY {logg0} LTE\nTITLE  [{MH_g0}] VTURB={vturb0}  L/H=1.25 NOVER NEW ODF\n"
        header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g0):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
        tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
        def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
            with open(input_file, 'r') as f_in:
                lines = f_in.readlines()

            # Remove the first 3 rows and the last 2 rows
            lines = lines[3:-2]

            with open(output_file, 'w') as f_out:
                f_out.write(header1)
                f_out.write(header2)
                f_out.writelines(lines)
                f_out.write(tail)

        read_and_copy_file('modelatmosphere.txt', 'out.txt')
        comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
        os.system(comandoATMOS)
        preparaAbundance(nome_Stella)
        os.chdir(path_to_abundance)
        creaParamfileAbnd(vturbo(T, logg, MH_g))
        SlanciaAbundance(2)
        nome_output = f"{nome_Stella}_INT{MH_g:.2f}_{T:.0f}_{logg:.2f}"
        comando_bsh2 = f"mv PINOLOS {path_to_output}/{nome_output}"
        os.system(comando_bsh2)
        return lista_nome_output
def sposta(nome_oggetto, path_oggetto, path_destinazione, nome_oggetto_destinazione):
    in_dir = os.getcwd()
    if path_oggetto[-1] == "/":
        path_obj = path_oggetto[:-1]
    else:
        path_obj = path_oggetto 
    if path_destinazione[-1] == "/":
        path_D = path_destinazione[:-1]
    else:
        path_D = path_destinazione
    COMANDO = f"mv {path_obj}/{nome_oggetto} {path_D}/{nome_oggetto_destinazione}"
    os.system(COMANDO)
    os.chdir(in_dir)
def copia(nome_oggetto, path_oggetto, path_destinazione, nome_oggetto_destinazione):
    in_dir = os.getcwd()
    if path_oggetto[-1] == "/":
        path_obj = path_oggetto[:-1]
    else:
        path_obj = path_oggetto 
    if path_destinazione[-1] == "/":
        path_D = path_destinazione[:-1]
    else:
        path_D = path_destinazione
    COMANDO = f"cp {path_obj}/{nome_oggetto} {path_D}/{nome_oggetto_destinazione}"
    os.system(COMANDO)
    os.chdir(in_dir)
def getTail(T, L, M):
    file_path = path_to_atmosphere + f"MH{M:.1f}/atmoT{int(T)}{L:.1f}.txt"
    # Leggi l'ultima riga del file
    with open(file_path, 'r') as file:
        last_line = file.readlines()[-2]
    # Estrai i numeri dalla riga usando una regex
    numbers = re.findall(r'[-+]?\d*\.\d+|[-+]?\d+', last_line)
    # Converti i numeri estratti in float
    numbers = [float(num) for num in numbers]
    numbers[0]

    if numbers[1] == 0.0:
        grandezza = "E+00"
    if numbers[1] < 0:
        grandezza = f"E-0{int(abs(numbers[1]))}"
    if numbers[1] > 0:
        grandezza = f"E+0{int(numbers[1])}"

    returner = f"{numbers[0]}" + grandezza
    return returner
def vturbo(T, LG, Met):
    return 0.14 - 0.08 * (Met) + 4.9 * T / 10000 - 0.47 * LG
def creaParamfileAbnd(microturb, numero_abundance=0, path_to_abundance=path_to_abundance, file_linee="stellaFes"):
    if numero_abundance==0:
        with open('param.txt', 'w') as paramfile:
            paramfile.write(f'ATMname ModelE\nLINESname {file_linee}\nOUTname PINOLOS\nvturb {str(microturb)}')
            sposta('param.txt', os.getcwd(), path_to_abundance, 'param.txt')
    if numero_abundance==1:
        with open('param.txt', 'w') as paramfile:
            paramfile.write(f'ATMname ModelE\nLINESname {file_linee}\nOUTname PINOLOS1\nvturb {str(microturb)}')
            sposta('param.txt', os.getcwd(), path_to_abundance_0, 'param.txt')
    if numero_abundance==2:
        with open('param.txt', 'w') as paramfile:
            paramfile.write(f'ATMname ModelE\nLINESname {file_linee}\nOUTname PINOLOS2\nvturb {str(microturb)}')
            sposta('param.txt', os.getcwd(), path_to_abundance_1, 'param.txt')
    if numero_abundance==3:
        with open('param.txt', 'w') as paramfile:
            paramfile.write(f'ATMname ModelE\nLINESname {file_linee}\nOUTname PINOLOS3\nvturb {str(microturb)}')
            sposta('param.txt', os.getcwd(), path_to_abundance_2, 'param.txt')
def SlanciaAbundance(n_istanze=1):
    if (n_istanze >= 1) and (n_istanze <= 4): 
        if n_istanze==1:
            DIR_INI = os.getcwd()
            os.chdir(path_to_abundance)
            abnd_path = "./abundance"
            process1 = subprocess.Popen([abnd_path, '-t'])
            process1.wait()
            os.chdir(DIR_INI)
        if n_istanze==2:
            DIR_INI = os.getcwd()
            os.chdir(path_to_abundance)
            abnd_path = "./abundance"
            process1 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_0)
            process2 = subprocess.Popen([abnd_path, '-t'])
            # Attendi che entrambi i processi terminino
            process1.wait()
            process2.wait()
            os.chdir(DIR_INI)
        if n_istanze==3:
            DIR_INI = os.getcwd()
            abnd_path = "./abundance"
            os.chdir(path_to_abundance)
            process1 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_0)
            process2 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_1)
            process3 = subprocess.Popen([abnd_path, '-t'])
            # Attendi che entrambi i processi terminino
            process1.wait()
            process2.wait()
            process3.wait()
            os.chdir(DIR_INI)
        if n_istanze==4:
            DIR_INI = os.getcwd()
            abnd_path = "./abundance"
            os.chdir(path_to_abundance)
            process1 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_0)
            process2 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_1)
            process3 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_2)
            process4 = subprocess.Popen([abnd_path, '-t'])
            # Attendi che entrambi i processi terminino
            process1.wait()
            process2.wait()
            process3.wait()
            process3.wait()
            os.chdir(DIR_INI)
    else:
        print(f"Numero di istanze assegnate DEVE essere compreso tra 1 e 4 inclusi. Inserito {n_istanze}")
        return None
def indici_elementiuguali(lista1, lista2):
    indici = [i for i, (x, y) in enumerate(zip(lista1, lista2)) if x == y]
    return indici
def folderLog(path, mode=1):
    '''Restituisce una lista con i parametri delle atmosfere vincenti'''
    old_dir = os.getcwd()
    os.chdir(path)
    a = os.listdir()
    b = []
    returner = np.zeros(8)
    for i in range(len(a)):
        if a[i] != "log.txt" and a[i] != "exit":
            b.append(a[i]) #in b i nomi delle stelle
    H = np.zeros((len(b), 8)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
    min = 10
    j=0
    with open("log.txt", 'w') as f:
        f.write("    T      G      M      FeI            FeII           FeI-M/H        FeI-FeII       s(FeI)    \n")
        for i in range(len(b)):
            tempD = simpleAnalysisSingle(path, b[i], 26.1)
            tempS = simpleAnalysisSingle(path, b[i], 26.0)
            H[i][0] = tempS[1]
            H[i][1] = float(tempS[0])
            H[i][2] = float(tempD[0])
            if mode == 1:
                H[i][3] = (sparseFileName(path, b[i]))[1]
                H[i][4] = (sparseFileName(path, b[i]))[2]
                H[i][5] = (sparseFileName(path, b[i]))[0]
            if mode == 2:
                H[i][3] = (sparseFileName2(path, b[i]))[1]
                H[i][4] = (sparseFileName2(path, b[i]))[2]
                H[i][5] = (sparseFileName2(path, b[i]))[0]
            H[i][6] = tempS[2]
            H[i][7] = tempD[2]
            f.write(f"{H[i][3]:.0f}   {H[i][4]:.2f}   {H[i][5]:.2f}   {H[i][1]:.3}+-{H[i][6]:.3f}   {H[i][2]:.3}+-{H[i][7]:.3f}   {H[i][1]-H[i][5]:.4f}         {H[i][1]-H[i][2]:.3f}          {H[i][0]:.4f}\n")
    winners = []
    for i in range(len(b)): #erano 0.6                            0.21                    0.11
        if (abs(H[i][1]-H[i][5]) < 0.17) and (abs(H[i][1]-H[i][2])<0.01) and (abs(H[i][0])<0.005):
            winners.append([H[i][3], H[i][4], H[i][6], 100])
    ds=0.002
    dg=0.04
    dm=0.08
    q=1
    for u in range(100):
        for i in range(len(b)):
            if len(winners) < 4:
                if (abs(H[i][1]-H[i][5])<0.1+dm) and (abs(H[i][1]-H[i][2])<0.01+dg) and (abs(H[i][0])<0.005+ds):
                    if 100-5*q <= -700:
                        winners.append([H[i][3], H[i][4], H[i][5], 10/q])
                    else:
                        winners.append([H[i][3], H[i][4], H[i][5], 100/q])#-5*(q)])
            q=q+1
            ds=ds+0.001
            dg=dg+0.005
            dm=dm+0.01
    os.chdir(old_dir)
    return winners
def tra_le_atmosfere(path_to_output=path_to_output, nome_stella='HARPN'):
    '''la miglior atmosfera, si restituisce una lista contenente le '''
    a=[]
    # findBestAtmosphereinFolder(path_to_output+f"HARPN{I}.fits", "A")
    for I in range(1, N_stelle+1):
        a.append(folderLog(path_to_output+f"{nome_stella}{I-1}.fits"))
    return a
def intermediateATM(atmospheric_param):
    RET = []
    return_param = np.zeros(3)
    if len(atmospheric_param) == 0:
        print("nessuna atmosfera passata!")
        return None
    elif len(atmospheric_param) == 1:
        return atmospheric_param[0]
    elif len(atmospheric_param) == 2:
        for i in range(3):
            return_param[i] = (1.0*atmospheric_param[0][i]+1.0*atmospheric_param[1][i])/2
        RET.append(return_param[0])
        RET.append(return_param[1])
        RET.append(return_param[2])
        return RET
    elif len(atmospheric_param) == 3:
        A = []
        A.append((atmospheric_param[0][0]+atmospheric_param[1][0]+atmospheric_param[2][0])/3)
        A.append((atmospheric_param[0][1]+atmospheric_param[1][1]+atmospheric_param[2][1])/3)
        A.append((atmospheric_param[0][2]+atmospheric_param[1][2]+atmospheric_param[2][2])/3)
        return A
    elif len(atmospheric_param) == 4:
        A = []
        normaliz = (atmospheric_param[0][3]+atmospheric_param[1][3]+atmospheric_param[2][3]+atmospheric_param[3][3]) #media ponderata sui pesi, a seconda di quanto è buona l'atmosfera
        A.append((atmospheric_param[0][0]*atmospheric_param[0][3]+atmospheric_param[1][0]*atmospheric_param[1][3]+atmospheric_param[2][0]*atmospheric_param[2][3]+atmospheric_param[3][0]*atmospheric_param[3][3])/normaliz)
        A.append((atmospheric_param[0][1]*atmospheric_param[0][3]+atmospheric_param[1][1]*atmospheric_param[1][3]+atmospheric_param[2][1]*atmospheric_param[2][3]+atmospheric_param[3][1]*atmospheric_param[3][3])/normaliz)
        A.append((atmospheric_param[0][2]*atmospheric_param[0][3]+atmospheric_param[1][2]*atmospheric_param[1][3]+atmospheric_param[2][2]*atmospheric_param[2][3]+atmospheric_param[3][2]*atmospheric_param[3][3])/normaliz)
        return A
    elif len(atmospheric_param) == 5:
        A = []
        normaliz = (atmospheric_param[0][3]+atmospheric_param[1][3]+atmospheric_param[2][3]+atmospheric_param[3][3]+atmospheric_param[4][3]) #media ponderata sui pesi, a seconda di quanto è buona l'atmosfera
        A.append((atmospheric_param[0][0]*atmospheric_param[0][3]+atmospheric_param[1][0]*atmospheric_param[1][3]+atmospheric_param[2][0]*atmospheric_param[2][3]+atmospheric_param[3][0]*atmospheric_param[3][3]+atmospheric_param[4][0]*atmospheric_param[4][3])/normaliz)
        A.append((atmospheric_param[0][1]*atmospheric_param[0][3]+atmospheric_param[1][1]*atmospheric_param[1][3]+atmospheric_param[2][1]*atmospheric_param[2][3]+atmospheric_param[3][1]*atmospheric_param[3][3]+atmospheric_param[4][1]*atmospheric_param[4][3])/normaliz)
        A.append((atmospheric_param[0][2]*atmospheric_param[0][3]+atmospheric_param[1][2]*atmospheric_param[1][3]+atmospheric_param[2][2]*atmospheric_param[2][3]+atmospheric_param[3][2]*atmospheric_param[3][3]+atmospheric_param[4][2]*atmospheric_param[4][3])/normaliz)
        return A
    else:
        A = [4750, 1.5, -1.0]
def stampaAnalisi(path_file, nome):
    A = simpleAnalysisSingle(path_file, nome, 26.0)
    B = simpleAnalysisSingle(path_file, nome, 26.1)
    with open("exit", 'w') as f:
        f.write(f"FeI = {A[0]}+{A[2]}\nFeII = {B[0]}+{B[2]}\n s(FeI) = {A[1]}\nDF = {A[0]-B[0]}")
    sposta("exit", os.getcwd(), path_file, "exit")
def simpleAnalysisSingle(path_file, nome_file, elemento=26.0): #passa path e nome file, restituisce abbondanza di un elemento, slope della retta che interpola i punti sul grafico abbondanza-energia potenziale, restituisce anche la standard deviation
    """Restituisce: media, slope, std"""
    DIR_INI = os.getcwd()
    os.chdir(path_file)
    returner = np.zeros(3)
    #t1 = Table.read('stellaFes', format='ascii')
    print(os.getcwd())
    t2 = Table.read(nome_file, format='ascii')
    x=t2['col3']
    y=t2['col7']
    fe=t2['col2']
    limit=((fe==elemento))
    x=x[limit]
    y=y[limit]
    limit=((y<np.mean(y)+1.5*np.std(y))&(y>np.mean(y)-1.5*np.std(y)))
    x=x[limit]
    y=y[limit]
    popt, pcov = curve_fit(linear_func, x, y,method='lm')
    std1=np.std(y-(linear_func(x,*popt)))
    #print(f"deviazione standard = {std1}")
    os.chdir(DIR_INI)
    returner[0] = np.mean(y) #media
    returner[1] = popt[0]    #slope
    returner[2] = std1       #std
    print(nome_file)
    return returner
def simpleAnalysisDouble(path_file, nome_file, elemento1=26.0, elemento2=26.1): #passa path del file, il nome, e restituisce le abbondanze di due elementi (o diverse ionizzazioni)
    """Restituisce: abbondanza el1, abbondanza el2"""
    DIR_INI = os.getcwd()
    os.chdir(path_file)
    returner = np.zeros(2)
    #t1 = Table.read('stellaFes', format='ascii')
    t2 = Table.read(nome_file, format='ascii')
    t3 = Table.read(nome_file, format='ascii')
    x2=t2['col3']
    y2=t2['col7']
    fe2=t2['col2']
    limit=((fe2==elemento1))
    x2=x2[limit]
    y2=y2[limit]
    limit=((y2<np.mean(y2)+1*np.std(y2))&(y2>np.mean(y2)-1*np.std(y2)))
    x2=x2[limit]
    y2=y2[limit]
    x3=t3['col3']
    y3=t3['col7']
    fe3=t3['col2']
    limit=((fe3==elemento2))
    x3=x3[limit]
    y3=y3[limit]
    limit=((y3<np.mean(y3)+1*np.std(y3))&(y3>np.mean(y3)-1*np.std(y3)))
    x3=x3[limit]
    y3=y3[limit]
    os.chdir(DIR_INI)
    returner[0] = np.mean(y2) #abbondanza elemento1
    returner[1] = np.mean(y3)+correzione(returner[0]) #abbondanza elemento2
    return returner
def findAbundance(tempera, logG, metal, label, multiple_elements=0,  elemento1=26.0, elemento2=26.1):
    """Restituisce: multiple element==0 media, slope, std, multiple element==1 media el1 e media el2"""
    copia(f"atmoT{int(tempera)}{logG}.txt", path_to_atmosphere + f"MH{metal}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
    creaParamfileAbnd(vturbo(tempera, logG, metal))  #fai paramfile di abundance
    SlanciaAbundance()                                    #lancia abundance
    sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{metal}_{int(tempera)}_{logG}") #copia l'output di abundance nella cartella degli output
    if multiple_elements == 0:
        results = simpleAnalysisSingle(path_to_output, f"TSTELLA{metal}_{int(tempera)}_{logG}", elemento1)
    else:
        results = simpleAnalysisDouble(path_to_output, f"GSTELLA{metal}_{int(tempera)}_{logG}", elemento1, elemento2)
    return results
def iterative_temperatureSearch(tempera, logG, metal, trials=8, elemento=26.0, controller=1):
    diarioTemp = np.zeros((trials, 4))
    i=0
    min = 1000
    control = 1
    while controller==1 and i < trials:
        print(f"trial = {i}")
        file_check = f"{path_to_output}/TSTELLA{metal}_{int(tempera)}_{logG}"
        file_checkG = f"{path_to_output}/GSTELLA{metal}_{int(tempera)}_{logG}"
        if (exists(file_check)):
            break
        # if (exists(file_checkG)) and control==1:
        #     if simpleAnalysisSingle(os.getcwd(), file_checkG)[1] > 0:
        #         tempera=tempera+250
        #     if simpleAnalysisSingle(os.getcwd(), file_checkG)[1] < 0:
        #         tempera=tempera-250
        #     control = 0
        A = findAbundance(tempera, logG, metal, "T", 0, elemento)
        diarioTemp[i][0] = A[0]
        diarioTemp[i][1] = A[1]
        diarioTemp[i][2] = A[2]
        diarioTemp[i][3] = tempera
        if (i>=1) and (abs(diarioTemp[i][1])>abs(diarioTemp[i-1][1])):
            break
        metal = find_closest(A[0], metallic)
        required_index = 0
        for k in range(trials):
            if (k!=i) and (diarioTemp[k][3]==diarioTemp[i][3]):
                controller = 0
        if ((tempera > 3499) and (tempera < 12750)): 
            if (diarioTemp[i][1]>0):
                tempera = tempera + 250
                tempera = int(tempera)
            else:
                if tempera > 3749:
                    tempera = tempera - 250
                    tempera = int(tempera)
        i=i+1
    if controller == 1:
        print("il numero di trials non è stato sufficiente ad ottenere un risultato, prova con un numero di trials maggiore")
        if i != 0:
            for j in range(trials):
                if (diarioTemp[j][3] != 0) and (abs(diarioTemp[j][1]) < abs(min)): #!= 0 perché viene inizializzato con alcuni 0 che potrebbero non essere utilizzati, in tal caso ci si ferma prima. Si cerca la miglior slope, infatti si ricerca min e via via si sostituisce con la slope più bassa. Alla slope più bassa, la j-esima, corrisponde la j-esima atmosfera
                    min = diarioTemp[j][1]
                    required_index = j
            #print(f"la temperatura {diarioTemp[required_index][3]} è il valore più vicino a quello cercato \ncorrispondente a una slope di {diarioTemp[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioTemp[required_index][0]} +- {diarioTemp[required_index][2]}")
            returner = np.zeros(4)
            returner[0] = diarioTemp[required_index][0] #media
            returner[1] = diarioTemp[required_index][1] #slope
            returner[2] = diarioTemp[required_index][2] #std
            returner[3] = diarioTemp[required_index][3] #temperatura
            return returner
    if controller == 0:
        for j in range(trials):
            if (diarioTemp[j][3] != 0) and (abs(diarioTemp[j][1]) < abs(min)): #!= 0 perché viene inizializzato con alcuni 0 che potrebbero non essere utilizzati, in tal caso ci si ferma prima. Si cerca la miglior slope, infatti si ricerca min e via via si sostituisce con la slope più bassa. Alla slope più bassa, la j-esima, corrisponde la j-esima atmosfera
                min = diarioTemp[j][1]
                required_index = j
        print(f"la temperatura {diarioTemp[required_index][3]} è il valore più vicino a quello cercato \ncorrispondente a una slope di {diarioTemp[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioTemp[required_index][0]} +- {diarioTemp[required_index][2]}")
        returner = np.zeros(4)
        returner[0] = diarioTemp[required_index][0] #media
        returner[1] = diarioTemp[required_index][1] #slope
        returner[2] = diarioTemp[required_index][2] #std
        returner[3] = diarioTemp[required_index][3] #temperatura
        return returner
def iterative_logGSearch(tempera, logG, metal, trials=8, elemento=26.0, controller=1):
    diarioLogG = np.zeros((trials, 3))
    diarioLogG = diarioLogG + 555.9
    i=0
    min = 1000
    while controller==1 and i<trials:
        print(f"trial = {i}")
        file_check = f"{path_to_output}/GSTELLA{metal}_{int(tempera)}_{logG}"
        file_checkT = f"{path_to_output}/TSTELLA{metal}_{int(tempera)}_{logG}"
        if (exists(file_check)):
            break
        # if (exists(file_checkT)):
        #     if simpleAnalysisDouble(os.getcwd(), file_checkT)[0]-simpleAnalysisDouble(os.getcwd(), file_checkT)[1] > 0:
        #         logG=logG+0.5
        #     if simpleAnalysisDouble(os.getcwd(), file_checkT)[0]-simpleAnalysisDouble(os.getcwd(), file_checkT)[1] < 0:
        #         logG=logG-0.5
        outPut = findAbundance(tempera, logG, metal, "G", 1, 26.0, 26.1)
        diarioLogG[i][0] = outPut[0]
        diarioLogG[i][1] = outPut[1]
        diarioLogG[i][2] = logG
        required_index = 0
        for k in range(trials):
            if (k!=i) and (diarioLogG[k][2]==diarioLogG[i][2]):
                controller = 0
        if ((logG > -0.1) and (logG < 5.1)): 
            if (diarioLogG[i][0] - diarioLogG[i][1] > 0):
                logG = logG + 0.5
            else:
                if logG > 0.4:
                    logG = logG - 0.5
        print(f"{diarioLogG[i][0]-diarioLogG[i][1]} con {diarioLogG[i][2]}")
        i=i+1
    if controller == 1:
        if i != 0:
            for j in range(i):
                if abs(diarioLogG[j][1]-diarioLogG[j][0]) < abs(min):
                    min = (diarioLogG[j][0]-diarioLogG[j][1])
                    required_index = j
            print(f"il logG {diarioLogG[required_index][2]} è il valore più vicino a quello cercato \ncorrispondente a elemento 26.1 {diarioLogG[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioLogG[required_index][0]}")
            returner = np.zeros(3)
            returner[0] = diarioLogG[required_index][0] #abbondanza elemento 1
            returner[1] = diarioLogG[required_index][1] #abbondanza elemento 2
            returner[2] = diarioLogG[required_index][2] #logG cercata
            return returner
    if controller == 0:
        for j in range(i):
            if abs(diarioLogG[j][1]-diarioLogG[j][0]) < abs(min):
                min = (diarioLogG[j][0]-diarioLogG[j][1])
                required_index = j
        print(f"il logG {diarioLogG[required_index][2]} è il valore più vicino a quello cercato \ncorrispondente a elemento 26.1 {diarioLogG[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioLogG[required_index][0]}")
        returner = np.zeros(3)
        returner[0] = diarioLogG[required_index][0] #abbondanza elemento 1
        returner[1] = diarioLogG[required_index][1] #abbondanza elemento 2
        returner[2] = diarioLogG[required_index][2] #logG cercata
        return returner                                 #Iron_lines#al posto di molto_linee per il ferro
def creaOptFileARES(specfits='Stello.fits', readlinedat='molte_linee', fileout='stella.ares', lambdai="4600.", lambdaf="6700.", smoothder="4", space="3.0", rejt="1;5764,5766,6047,6052,6068,6076", lineresol="0.1", miniline="2", plots_flag="0", rvmask="3,6021.8,6024.06,6027.06,6024.06,20"):
    with open('mine.opt', 'w') as paramfile:
        paramfile.write(f'specfits=\'{specfits}\'\nreadlinedat=\'{readlinedat}\'\nfileout=\'{fileout}\'\nlambdai={lambdai}\nlambdaf={lambdaf}\nsmoothder={smoothder}\nspace={space}\nrejt={rejt}\nlineresol={lineresol}\nminiline={miniline}\nplots_flag={plots_flag}\nrvmask=\'{rvmask}\'')
def parameterAnalysis(param_space_directions, T_center, log_center, M_center, T_steps=50, log_steps=0.1, M_steps=0.1, T_tot=8, log_tot=8, M_tot=10):
    if (param_space_directions < 1) or (param_space_directions>4):
        print("Il primo argomento in parameterAnalysis deve essere un int \n1 - muoviti solo in temperatura\n2 - muoviti in temperatura e logg\n3 - muoviti in temperatura logg e metallicità\n4 muoviti solo in logg")
    elif param_space_directions == 1:
        T = T_center - (T_steps*T_tot)/2
        T_temp = T
        ### muovi temperatura
        for i in range(T_tot):
            crea_ATMinterpol(round(T_temp), round(log_center, 1), round(M_center, 2))                
            T_temp = T_temp + T_steps
    elif param_space_directions == 2:
        T = T_center - (T_steps*T_tot)/2
        log = log_center - (log_steps*log_tot)/2
        T_temp = T
        log_temp = log
        ### muovi temperatura e al suo interno il logg
        for i in range(T_tot):
            for j in range(log_tot):
                crea_ATMinterpol(round(T_temp), round(log_temp, 1), round(M_center, 2))
                log_temp = log_temp + log_steps
            T_temp = T_temp + T_steps
            log_temp = log
    elif param_space_directions == 3:
        T = T_center - (T_steps*T_tot)/2
        log = log_center - (log_steps*log_tot)/2
        M = M_center - (M_steps*T_tot)/2
        T_temp = T
        log_temp = log
        M_temp = M
        ### muovi temperatura e al suo interno il logg
        for i in range(T_tot):
            for j in range(log_tot):
                for k in range(M_tot):
                    crea_ATMinterpol(round(T_temp), round(log_temp, 1), round(M_temp, 2))
                    M_temp = M_temp + M_steps
                log_temp = log_temp + log_steps
                M_temp = M
            T_temp = T_temp + T_steps
            log_temp = log
    print("FINITO")
def find_closest(num, arr):
    arr = np.array(arr)
    closest_index = np.abs(arr - num).argmin()
    return arr[closest_index]
def write_file(output_file, header1, header2, lines, tail):
        with open(output_file, 'w') as f_out:
            f_out.write(header1)
            f_out.write(header2)
            f_out.writelines(lines)
            f_out.write(tail)
def rimuoviHeader_atm(input_file, input_path, output_file, output_path):
    A=(os.getcwd())
    os.chdir(input_path)
    with open(input_file, 'r') as filein:
        lines = filein.readlines()
    lines = lines[23:-2]
    os.chdir(output_path)
    with open(output_file, 'w') as fileout:
        fileout.writelines(lines)
    os.chdir(A)
def SinterpolaT(T, logg, M, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    t_pradk = find_closest(T, T_EFF_Nint)
    T=find_closest(T, T_EFF_int) #temperatura più fine (25K) rispetto a logg e M qua sotto (che ancora sono su griglia vanilla)
    logg=find_closest(logg, LOGG_Nint)
    M=find_closest(M, MH_Nint)
    vturbs = vturbo(T, logg, M)
    header1 = f"TEFF   {T}.  GRAVITY {logg:.1f} LTE\nTITLE  [{M:.1f}] VTURB={vturbs}  L/H=1.25 NOVER NEW ODF\n"
    header2 = " OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   0.10000 ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    os.chdir(path_to_atmosphere + f"MH{M:.1f}")
    tail = f"PRADK {getTail(t_pradk, logg, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    for i in range(len(T_EFF_Nint)):
        nomeATMint = f'T{T_EFF_Nint[i]:.0f}{logg:.1f}.int'
        copia(f'atmoT{T_EFF_Nint[i]}{logg:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd())
        P.append(Table.read(f'T{T_EFF_Nint[i]:.0f}{logg:.1f}.int', format='ascii'))
    y = np.zeros((72, 10))
    for i in range(72):
        for j in range(10):
            a=[]
            for k in range(len(T_EFF_Nint)):
                a.append((P[k])[i][j])
            z = np.polyfit(T_EFF_Nint, np.asarray(a), 8)
            f = np.poly1d(z)
            x_new = np.linspace(T_EFF_int[0], T_EFF_int[-1], len(T_EFF_Nint)*1000)
            y_new = f(x_new)
            T = find_closest(T, x_new)
            indice = np.where(x_new == T)[0]
            y[i][j] = y_new[indice]
            if y[i][j]<0:
                y[i][j] = 0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'atm{T:.0f}_{logg}.int'
    write_file( nome_out, header1, header2, linea, tail)
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return P
def interpolaT(T, logg, M, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    t_pradk = find_closest(T, T_EFF_Nint)
    T=find_closest(T, T_EFF_int) #temperatura più fine (25K) rispetto a logg e M qua sotto (che ancora sono su griglia vanilla)
    logg=find_closest(logg, LOGG_Nint)
    M=find_closest(M, MH_Nint)
    vturbs = vturbo(T, logg, M)
    header1 = f"TEFF   {T}.  GRAVITY {logg:.1f} LTE\nTITLE  [{M:.1f}] VTURB={vturbs}  L/H=1.25 NOVER NEW ODF\n"
    header2 = " OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   0.10000 ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    os.chdir(path_to_atmosphere + f"MH{M:.1f}")
    tail = f"PRADK {getTail(t_pradk, logg, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    for i in range(len(T_EFF_Nint)):
        nomeATMint = f'T{T_EFF_Nint[i]:.0f}{logg:.1f}.int'
        copia(f'atmoT{T_EFF_Nint[i]}{logg:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd())
        P.append(Table.read(f'T{T_EFF_Nint[i]:.0f}{logg:.1f}.int', format='ascii'))
    y = np.zeros((72, 10))
    o=-1
    for i in range(72):
        for j in range(10):
            A=[]
            if (find_closest(t_pradk, T_EFF_Nint) == 3500):
                A.append((P[0])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[1])[i][j])
                A.append((P[2])[i][j])
                z = np.polyfit([T_EFF_Nint[0],T_EFF_Nint[1]], np.asarray(A), 2)         
            elif (find_closest(t_pradk, T_EFF_Nint) == 6000):
                A.append((P[-1])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[-2])[i][j])
                A.append((P[-3])[i][j])
                z = np.polyfit([T_EFF_Nint[-1],T_EFF_Nint[-2]], np.asarray(A), 2)  
            elif (find_closest(t_pradk, T_EFF_Nint) > 3500) and (find_closest(t_pradk, T_EFF_Nint) < 6000):
                for k in range(len(T_EFF_Nint)): #scorro su atmosfere
                    if (T > T_EFF_Nint[k-1]) and (T <= T_EFF_Nint[k]):
                        A.append((P[k-1])[i][j])
                        A.append((P[k])[i][j])
                        A.append((P[k+1])[i][j])
                        z = np.polyfit([T_EFF_Nint[k-1],T_EFF_Nint[k],T_EFF_Nint[k+1]], np.asarray(A), 2)
                        m=(A[1]-A[0])/(T_EFF_Nint[k]-T_EFF_Nint[k-1])
                        o=k-1
            f = np.poly1d(z)
            
            x_new = np.linspace(T_EFF_int[0], T_EFF_int[-1], len(T_EFF_int)*1000)
            y_new = f(x_new)
            T = find_closest(T, x_new)
            indice = np.where(x_new == T)[0]
            y[i][j] = y_new[indice]
            if y[i][j]<0:
                y[i][j] = 0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'atm{T:.0f}_{logg}.int'
    write_file( nome_out, header1, header2, linea, tail)
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return P
def interpolaM(T, logg, M, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    Mi=M
    t_pradk = find_closest(T, T_EFF_Nint)
    T=find_closest(T, T_EFF_Nint) #temperatura più fine (25K) rispetto a logg e M qua sotto (che ancora sono su griglia vanilla)
    logg=find_closest(logg, LOGG_Nint)
    Mint=find_closest(M, MH_int)
    M=find_closest(M, MH_Nint)
    vturbs = vturbo(T, logg, M)
    header1 = f"TEFF   {T}.  GRAVITY {logg:.1f} LTE\nTITLE  [{M:.1f}] VTURB={vturbs}  L/H=1.25 NOVER NEW ODF\n"
    header2 = " OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   0.10000 ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    tail = f"PRADK {getTail(t_pradk, logg, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    for i in range(len(MH_Nint)):
        os.chdir(path_to_atmosphere + f"MH{MH_Nint[i]:.1f}")
        nomeATMint = f'{MH_Nint[i]}T{T:.0f}{logg:.1f}.int'
        copia(f'atmoT{T}{logg:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd())
        P.append(Table.read(nomeATMint, format='ascii'))
    y = np.zeros((72, 10))
    o=-1
    for i in range(72):
        for j in range(10):
            A=[]
            if (find_closest(M, MH_Nint) == 0.5):
                A.append((P[0])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[1])[i][j])
                A.append((P[2])[i][j])
                z = np.polyfit([MH_Nint[0],MH_Nint[1]], np.asarray(A), 2)         
            elif (find_closest(M, MH_Nint) == -2.5):
                A.append((P[-1])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[-2])[i][j])
                A.append((P[-3])[i][j])
                z = np.polyfit([MH_Nint[-1],MH_Nint[-2]], np.asarray(A), 2)  
            elif (find_closest(M, MH_Nint) > -2.5) and (find_closest(M, MH_Nint) < 0.5):
                for k in range(len(MH_Nint)): #scorro su atmosfere
                    if (Mint <= MH_Nint[k-1]) and (Mint > MH_Nint[k]):
                        A.append((P[k-1])[i][j])
                        A.append((P[k])[i][j])
                        A.append((P[k+1])[i][j])
                        z = np.polyfit([MH_Nint[k-1],MH_Nint[k],MH_Nint[k+1]], np.asarray(A), 2)
                        m=(A[1]-A[0])/(MH_Nint[k]-MH_Nint[k-1])
                        o=k-1
            f = np.poly1d(z)
            
            x_new = np.linspace(MH_int[0], MH_int[-1], len(MH_int)*1000)
            y_new = f(x_new)
            Mint = find_closest(Mint, x_new)
            indice = np.where(x_new == Mint)[0]
            #y_new = A[0]+(T-T_EFF_Nint[o])*m
            y[i][j] = y_new[indice]
            if y[i][j]<0:
                y[i][j] = 0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'{Mi:.2f}atm{T:.0f}_{logg}.int'
    write_file( nome_out, header1, header2, linea, tail)
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return P
def Sinterpolalog(T, logg, M, variazioneTemperatura, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    T_in=T
    T=find_closest(T, T_EFF_Nint)              #temperatura su griglia originale
    logg_pradk = find_closest(logg, LOGG_Nint) #per formattazione
    logg=find_closest(logg, LOGG_int)          #logg su griglia interpolata
    M=find_closest(M, MH_Nint)                 #M su griglia originale
    vturbs = vturbo(T, logg, M)                #calcola vturb
    Fe_H = creaHeader_metal(M)
    header1 = f"TEFF   {T_in:.0f}.  GRAVITY {logg:.1f} LTE\nTITLE  [{M:.1f}] VTURB={vturbs:.2f}  L/H=1.25 NOVER NEW ODF\n"
    header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {Fe_H:.3f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    tail = f"PRADK {getTail(T, logg_pradk, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    os.chdir(path_to_atmosphere + f"MH{M:.1f}") #vai nella cartella con le atmosfere della giusta metallicità
    for i in range(len(LOGG_Nint)):             #scorri tutte le atmosfere che hanno temperatura fissata sulla griglia cambiando il logg
        nomeATMint = f'T{T:.0f}{LOGG_Nint[i]:.1f}.int'
        copia(f'atmoT{T}{LOGG_Nint[i]:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd()) #prende solo la tabella 72x10
        P.append(Table.read(f'T{T:.0f}{LOGG_Nint[i]:.1f}.int', format='ascii')) #la metto in una lista, la i-esima atmosfera corrisponde a quella avente T=T_fissato_su_griglia con log(g_i)
    y = np.zeros((72, 10)) 
    for i in range(72): #scorro su righe
        for j in range(10): #scorro su colonne
            a=[]
            for k in range(len(LOGG_Nint)): #scorro su atmosfere
                a.append((P[k])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
            z = np.polyfit(LOGG_Nint, np.asarray(a), 6) #fit polinomiale di 6° grado (molto buono per metallicità circa solare, meno per bassa metallicità)
            f = np.poly1d(z) #f è un polinomio di 6° grado con coefficienti dati da z
            x_new = np.linspace(LOGG_Nint[0], LOGG_Nint[-1], len(LOGG_Nint)*1000) #spanno lo spazio del dominio (1000 volte più grande dell'originale)
            y_new = f(x_new) #creo l'immagine
            LoG = find_closest(logg, x_new) #prendo il logg come quello più vicino a uno dei punti x_new
            indice = np.where(x_new == LoG)[0] #l'indice di x_new_i corretto corrisponde a quello di y_i corretto
            if y[i][j]<0: #non deve mai succedere
                y[i][j]=0
            y[i][j] = y_new[indice] + variazioneTemperatura[i][j]
            if i>0:
                if y[i][j]<y[i-1][j]: #scendendo con le righe deve sempre crescere
                    y[i][j] = y[i-1][j] #se non cresce blocco. In realtà da provare con fit esponenziale oppure facendo media del j-esimo elemento di differenti i-esime righe
                    if (y[i-1][j]==0) and (y[i+1][j]) ==0:
                        y[i][j]=0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'atmL{T_in:.0f}_{logg:.2f}.int'
    write_file( nome_out, header1, header2, linea, tail)
    copia( nome_out, os.getcwd(), path_to_output, nome_out + "aiai")
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return nome_out
def interpolalog(T, logg, M, variazioneTemperatura, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    T_in=T
    T=find_closest(T, T_EFF_Nint)              #temperatura su griglia originale
    logg_pradk = find_closest(logg, LOGG_Nint) #per formattazione
    logg=find_closest(logg, LOGG_int)          #logg su griglia interpolata
    Mint=find_closest(M, MH_int)
    M=find_closest(M, MH_Nint)                 #M su griglia originale
    vturbs = vturbo(T, logg, M)                #calcola vturb
    Fe_H = creaHeader_metal(Mint)
    header1 = f"TEFF   {T_in:.0f}.  GRAVITY {logg:.1f} LTE\nTITLE  [{Mint:.1f}] VTURB={vturbs:.2f}  L/H=1.25 NOVER NEW ODF\n"
    header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {Fe_H:.3f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    tail = f"PRADK {getTail(T, logg_pradk, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    os.chdir(path_to_atmosphere + f"MH{M:.1f}") #vai nella cartella con le atmosfere della giusta metallicità
    for i in range(len(LOGG_Nint)):             #scorri tutte le atmosfere che hanno temperatura fissata sulla griglia cambiando il logg
        nomeATMint = f'T{T:.0f}{LOGG_Nint[i]:.1f}.int'
        copia(f'atmoT{T}{LOGG_Nint[i]:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd()) #prende solo la tabella 72x10
        P.append(Table.read(f'T{T:.0f}{LOGG_Nint[i]:.1f}.int', format='ascii')) #la metto in una lista, la i-esima atmosfera corrisponde a quella avente T=T_fissato_su_griglia con log(g_i)
    y = np.zeros((72, 10)) 
    o=-1
    for i in range(72): #scorro su righe
        for j in range(10): #scorro su colonne
            A=[]
            if (find_closest(logg_pradk, LOGG_Nint) == 0.0):
                A.append((P[0])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[1])[i][j])
                A.append((P[2])[i][j])
                z = np.polyfit([LOGG_Nint[0],LOGG_Nint[1],LOGG_Nint[2]], np.asarray(A), 2)         
            elif (find_closest(logg_pradk, LOGG_Nint) == 5.0):
                A.append((P[-1])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[-2])[i][j])
                A.append((P[-3])[i][j])
                z = np.polyfit([LOGG_Nint[-1],LOGG_Nint[-2],LOGG_Nint[-3]], np.asarray(A), 2)  
            elif (find_closest(logg_pradk, LOGG_Nint) > 0.0) and (find_closest(logg_pradk, LOGG_Nint) < 5.0):
                for k in range(len(LOGG_Nint)): #scorro su atmosfere
                    if (logg >= LOGG_Nint[k-1]) and (logg <= LOGG_Nint[k]):
                        A.append((P[k-1])[i][j])
                        A.append((P[k])[i][j])
                        A.append((P[k+1])[i][j])
                        z = np.polyfit([LOGG_Nint[k-1],LOGG_Nint[k],LOGG_Nint[k+1]], np.asarray(A), 2) #fit polinomiale di 6° grado (molto buono per metallicità circa solare, meno per bassa metallicità)
            f = np.poly1d(z) #f è un polinomio di 6° grado con coefficienti dati da z
            # Y = np.asarray(A)
            # if find_closest(, LOGG_Nint)
            x_new = np.linspace(LOGG_Nint[0], LOGG_Nint[-1], len(LOGG_Nint)*1000) #spanno lo spazio del dominio (1000 volte più grande dell'originale)
            y_new = f(x_new) #creo l'immagine
            # params, cov = curve_fit(expo_func, X, Y, maxfev=1000)
            # a, b, c = params
            # y_new = expo_func(x_new, a, b, c)
            LoG = find_closest(logg, x_new) #prendo il logg come quello più vicino a uno dei punti x_new
            indice = np.where(x_new == LoG)[0] #l'indice di x_new_i corretto corrisponde a quello di y_i corretto
            if y[i][j]<0: #non deve mai succedere
                y[i][j]=0
            y[i][j] = y_new[indice] + variazioneTemperatura[i][j]
            if i>0:
                if y[i][j]<y[i-1][j]: #scendendo con le righe deve sempre crescere
                    y[i][j] = y[i-1][j] #se non cresce blocco. In realtà da provare con fit esponenziale oppure facendo media del j-esimo elemento di differenti i-esime righe
                    if (y[i-1][j]==0) and (y[i+1][j]) ==0:
                        y[i][j]=0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'atmL{T_in:.0f}_{logg:.2f}.int'
    write_file( nome_out, header1, header2, linea, tail)
    copia( nome_out, os.getcwd(), path_to_output, nome_out + "aiai")
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return nome_out
def differenza_INT_grid(temperats, loggheg, metallus, linea=[]):
    T_close = find_closest(temperats, T_EFF_Nint)
    logg_close=find_closest(loggheg, LOGG_Nint)
    M_close=find_closest(metallus, MH_Nint)
    rimuoviHeader_atm(f'atmoT{T_close}{logg_close:.1f}.txt', path_to_atmosphere+f"MH{M_close}", f'atmoT{T_close}{logg_close:.1f}.txt', path_to_output)
    os.chdir(path_to_output)
    tavola = Table.read(f'atmoT{int(T_close)}{logg_close:.1f}.txt', format='ascii')
    rimuoviHeader_atm(f'atm{find_closest(temperats, T_EFF_int)}_{logg_close:.1f}.int', path_to_output, f'atm{temperats:.0f}_{logg_close:.1f}.int', path_to_output)
    tavola1 = Table.read(f'atm{temperats:.0f}_{logg_close:.1f}.int', format='ascii')
    matrice_differenze = np.zeros((72, 10))
    for i in range(72):
        m0=tavola1[i][0]-tavola[i][0]
        m1=tavola1[i][1]-tavola[i][1]
        m2=tavola1[i][2]-tavola[i][2]
        m3=tavola1[i][3]-tavola[i][3]
        m4=tavola1[i][4]-tavola[i][4]
        m5=tavola1[i][5]-tavola[i][5]
        m6=tavola1[i][6]-tavola[i][6]
        m7=tavola1[i][7]-tavola[i][7]
        m8=tavola1[i][8]-tavola[i][8]
        m9=tavola1[i][9]-tavola[i][9]
        matrice_differenze[i][0]=m0
        matrice_differenze[i][1]=m1
        matrice_differenze[i][2]=m2
        matrice_differenze[i][3]=m3
        matrice_differenze[i][4]=m4
        matrice_differenze[i][5]=m5
        matrice_differenze[i][6]=m6
        matrice_differenze[i][7]=m7
        matrice_differenze[i][8]=m8
        matrice_differenze[i][9]=m9
        linea.append(f" {m0:.8E}   {m1:.1f} {m2:.3E} {m3:.3E} {m4:.3E} {m5:.3E} {m6:.3E} {m7:.3E} {m8:.3E} {m9:.3E}\n")
    write_file( "differenza", '', '', linea, '')
    return matrice_differenze
def differenza_INT_grid_M(temperats, loggheg, metallus, linea=[]):
    T_close = find_closest(temperats, T_EFF_Nint)
    logg_close=find_closest(loggheg, LOGG_Nint)
    M_close=find_closest(metallus, MH_Nint)
    rimuoviHeader_atm(f'atmoT{T_close}{logg_close:.1f}.txt', path_to_atmosphere+f"MH{M_close}", f'atmoT{T_close}{logg_close:.1f}.txt', path_to_output)
    os.chdir(path_to_output)
    tavola = Table.read(f'atmoT{int(T_close)}{logg_close:.1f}.txt', format='ascii')
    rimuoviHeader_atm(f'{metallus}atm{find_closest(temperats, T_EFF_Nint)}_{logg_close:.1f}.int', path_to_output, f'{metallus}atm{find_closest(temperats, T_EFF_Nint):.0f}_{logg_close:.1f}.int', path_to_output)
    tavola1 = Table.read(f'{metallus}atm{find_closest(temperats, T_EFF_Nint):.0f}_{logg_close:.1f}.int', format='ascii')
    matrice_differenze = np.zeros((72, 10))
    for i in range(72):
        m0=tavola1[i][0]-tavola[i][0]
        m1=tavola1[i][1]-tavola[i][1]
        m2=tavola1[i][2]-tavola[i][2]
        m3=tavola1[i][3]-tavola[i][3]
        m4=tavola1[i][4]-tavola[i][4]
        m5=tavola1[i][5]-tavola[i][5]
        m6=tavola1[i][6]-tavola[i][6]
        m7=tavola1[i][7]-tavola[i][7]
        m8=tavola1[i][8]-tavola[i][8]
        m9=tavola1[i][9]-tavola[i][9]
        matrice_differenze[i][0]=m0
        matrice_differenze[i][1]=m1
        matrice_differenze[i][2]=m2
        matrice_differenze[i][3]=m3
        matrice_differenze[i][4]=m4
        matrice_differenze[i][5]=m5
        matrice_differenze[i][6]=m6
        matrice_differenze[i][7]=m7
        matrice_differenze[i][8]=m8
        matrice_differenze[i][9]=m9
        linea.append(f" {m0:.8E}   {m1:.1f} {m2:.3E} {m3:.3E} {m4:.3E} {m5:.3E} {m6:.3E} {m7:.3E} {m8:.3E} {m9:.3E}\n")
    write_file( "differenza", '', '', linea, '')
    return matrice_differenze
def creaHeader_metal(Fe_H, metallic=metallic_header, ABUNDANCE_SCALE=ABUNDANCE_SCALE_header):
    return 10**Fe_H
def preparaAbundance(stellaFITS, path_to_abundance=path_to_abundance, eqw_low=5, eqw_high=100, lineList='molte_linee'):
    # Apre il file in modalità lettura
    """Nome del file FITS (doppler corretto) contenente la stella da analizzare
    eqw low and high sono gli estremi di equivalent width accettata nel processo di analisi, per trattare la curva di crescita delle eqw e rimanere fuori del rumore e dalla saturazione della riga"""
    i=0
    os.chdir(path_to_linemake)
              #Iron_lines#
    with open('molte_linee', 'r') as file:
        LINEM = repr(file.read())
    os.chdir(path_to_ARES)
    creaOptFileARES(stellaFITS, lineList)
    ! ./ARES
    with open('stella.ares', 'r') as file:
        ARES = repr(file.read())

    vect1 = LINEM.split("\\n")
    vect2 = ARES.split("\\n")
    VECT1 = np.zeros((len(vect1),9))
    VECT2 = np.zeros((len(vect2),9))

    for i in range(1,len(vect2)-1):
        temp2 = vect2[i].split()
        VECT2[i-1][0]= temp2[0]
        VECT2[i-1][1]= temp2[1]
        VECT2[i-1][2]= temp2[2]
        VECT2[i-1][3]= temp2[3]
        VECT2[i-1][4]= temp2[4]
        VECT2[i-1][5]= temp2[5]
        VECT2[i-1][6]= temp2[6]
        VECT2[i-1][7]= temp2[7]
        VECT2[i-1][8]= temp2[8]
    t4 = Table.read('stella.ares', format='ascii')

    for i in range(1, len(vect1)-1):
        temp1 = vect1[i].split()
        VECT1[i-1][0]= float(temp1[0])
        VECT1[i-1][1]= temp1[1]
        VECT1[i-1][2]= float(temp1[2])
        VECT1[i-1][3]= temp1[3]
        VECT1[i-1][4]= temp1[4]
        VECT1[i-1][5]= 1.0
        VECT1[i-1][6]= 99
        VECT1[i-1][7]= 0

    for i in range(len(VECT1)):
        for j in range(len(VECT2)):
            if (VECT1[i][0] == VECT2[j][0]) and (VECT2[j][4]>eqw_low) and (VECT2[j][4]<eqw_high):
                VECT1[i][7] = VECT2[j][4]

    # for i in range(1, len(vect1)-1): # per ARES linemake
    #     temp1 = vect1[i].split()
    #     VECT1[i-1][0]= float(temp1[0])
    #     VECT1[i-1][1]= temp1[1]
    #     VECT1[i-1][2]= float(temp1[2]) * 8066.01
    #     VECT1[i-1][3]= 0
    #     VECT1[i-1][4]= temp1[3]
    #     VECT1[i-1][5]= 1.0
    #     VECT1[i-1][6]= 99
    #     VECT1[i-1][7]= 0

    ############################## PREPARA FILE PER ABUNDANCE ################################

    os.chdir(path_to_abundance)
    bug = []
    o=0
    T_atmosold=0
    for i in range(len(VECT1)):
        if VECT1[i][7]>eqw_low and VECT1[i][7]<eqw_high: #if VECT1[i][1] == 26.0 and VECT1[i][7]>30:
            bug.append(VECT1[i][7])
            o=o+1
    # indici_righe_da_rimuovere = np.where((VECT1[:, 7] < eqw_low) | (VECT1[:, 7] > eqw_high))[0]
    # VECT1 = np.delete(VECT1, indici_righe_da_rimuovere, axis=0)
    linesout = 'stellantess'
    iron_linesout = 'stellaFes'
    with open(linesout, 'w') as file:
        for i in range(len(VECT1)):
            if VECT1[i][7]!=0:
                file.write(str(VECT1[i][0]))
                file.write('  ')
                file.write(str(VECT1[i][1]))
                file.write('  ')
                file.write(str(VECT1[i][2]))
                file.write('  ')
                file.write(str(VECT1[i][3]))
                file.write('  ')
                file.write(str(VECT1[i][4]))
                file.write('  ')
                file.write(str(VECT1[i][5]))
                file.write('  ')
                file.write(str(VECT1[i][6]))
                file.write('  ')
                file.write(str(VECT1[i][7]))
                file.write('  ')
                file.write('a')
                file.write('\n')

    # with open(iron_linesout, 'w') as file:
    #     for i in range(len(VECT1)):
    #         if VECT1[i][7]!=0 and (VECT1[i][1]==26 or VECT1[i][1]==26.1):
    #             file.write(str(VECT1[i][0]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][1]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][2]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][3]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][4]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][5]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][6]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][7]))
    #             file.write('  ')
    #             file.write('a')
    #             file.write('\n')
    with open(iron_linesout, 'w') as file:
        for i in range(len(VECT1)):
            if VECT1[i][7]!=0:
                file.write(str(VECT1[i][0]))
                file.write('  ')
                file.write(str(VECT1[i][1]))
                file.write('  ')
                file.write(str(VECT1[i][2]))
                file.write('  ')
                file.write(str(VECT1[i][3]))
                file.write('  ')
                file.write(str(VECT1[i][4]))
                file.write('  ')
                file.write(str(VECT1[i][5]))
                file.write('  ')
                file.write(str(VECT1[i][6]))
                file.write('  ')
                file.write(str(VECT1[i][7]))
                file.write('  ')
                file.write('a')
                file.write('\n')
    o=0
    vturb = 1.8
    comando_bash = f"cp {iron_linesout} {path_to_output}\{iron_linesout}"
    os.system(comando_bash)
    os.chdir(path_to_abundance)
def creaInterpolazione(temperat, loggh, metallu):
    interpolaT(temperat, loggh, metallu)
    T_close=find_closest(temperat, T_EFF_Nint)
    logg_close=find_closest(loggh, LOGG_Nint)
    M_close=find_closest(metallu, MH_Nint)
    differenze = differenza_INT_grid(temperat, loggh, metallu)
    interpolaM(temperat, loggh, metallu)
    differenze = differenze + differenza_INT_grid_M(temperat, loggh, metallu)
    b = interpolalog(temperat, loggh, metallu, differenze)
    return b
def abundance_interpolato(temperat, loggh, metallu, nome_stella, path_to_stella=path_to_output):
    olddir=os.getcwd()
    os.chdir(path_to_stella)
    preparaAbundance(nome_stella)
    B=creaInterpolazione(temperat, loggh, metallu)
    sposta(B, path_to_stella, path_to_abundance, "ModelE")
    os.chdir(path_to_abundance)
    creaParamfileAbnd(vturbo(temperat, loggh, metallu))
    SlanciaAbundance()
    os.chdir(path_to_stella)
    bashComm= "mkdir " + f"{nome_stella}"
    os.system(bashComm)
    path_to_output1=path_to_stella+f"{nome_stella}"
    nome_out = f"INT_{metallu}_{temperat}_{loggh}"
    sposta("PINOLOS", path_to_abundance, path_to_output1, nome_out)
    os.chdir(olddir)
    return nome_out
def avvicina_interpol(path_to_stella, nome_file, TEFF, G, M, nome_stella, nome_cartella=path_to_output):
    temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
    temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    sgn=1
    g=-1
    teff=-1
    met=1
    returner = []
    if abs(temp0[1]) > 0.001:
        if (abs(temp0[1]) < 0.06) and (abs(temp0[1]) >= 0.035):
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 70*sgn
        elif abs(temp0[1] > 0.01) and abs(temp0[1]) < 0.035:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 30*sgn
        elif abs(temp0[1]) < 0.01:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 10*sgn
        elif abs(temp0[1]) < 0.13 and abs(temp0[1]) > 0.06:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 150*sgn
        else:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 300*sgn
        crea_ATMinterpol(teff, G, temp0[0], nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{temp2:.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", path_to_output, path_to_output, f"{nome_file1}")
    else:
        nome_file1 = nome_file
        teff = TEFF
    temp0 = simpleAnalysisSingle(f"{path_to_output}", nome_file1, 26.0)
    temp1 = simpleAnalysisSingle(f"{path_to_output}", nome_file1, 26.1)
    sgn1=1
    if (temp0[0]-temp1[0])>0:
        sgn1=-1
    if abs(temp0[0]-temp1[0]) > 0.01:
        if abs(temp0[0]-temp1[0]) < 0.4:
            g = G+(temp0[0]-temp1[0])
        else:
            g = G+((temp0[0]-temp1[0]))
        crea_ATMinterpol(teff, g, temp0[0], nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{g:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{temp2:.2f}_{teff:.0f}_{g:.2f}"
        sposta(f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{g:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        #copia(f"{nome_file1}", path_to_output, path_to_output, f"{nome_file1}{nome_stella}")
        met=temp2
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    return returner
def avvicina_interpol2(path_to_stella, nome_file, temperatu, G, M, nome_stella, nome_cartella=path_to_output):
    temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
    temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    sgn=1
    g=-1.0
    teff=-1.0
    met=1.0
    DG = 0
    DT = 0
    if M-temp0[0] >= 0:
        dM = -(M - temp0[0])/3
    else:
        dM = (M - temp0[0])/3
    returner = []
    dsdg = -0.033574
    dsdt = -0.00023335
    dfdg = -0.5227
    dfdt = 0.001455
    f=-1*((temp0[0])-(temp1[0]))
    s = -1*temp0[1]
    if abs(temp0[1]) > 0.0002 or abs((temp0[0])-(temp1[0])) > 0.0005 or (M - temp0[0])>0.01:
        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
        DG = ((dsdt * f - dfdt * s)/(dfdg * dsdt - dsdg * dfdt))
        DT = ((dfdg * s - dsdg * f)/(dfdg * dsdt - dsdg * dfdt))
        if DG > 0.5:
            DG=0.5*(DG/abs(DG))
        if DT > 250:
            DT = 250*(DT/abs(DT))
        temperatu=temperatu+DT
        teff= temperatu/0.87
        G=G+DG
        crea_ATMinterpol(teff, G, temp0[0], nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{temp0[0]:.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        g=G
        met = temp0[0]
    else:
        nome_file1 = nome_file
        teff = temperatu
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    return returner
def avvicina_interpol3(path_to_stella, nome_file, temperatu, G, M, nome_stella, nome_cartella=path_to_output):
    temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
    temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    sgn=1
    g=-1.0
    teff=-1.0
    met=1.0
    DG = 0
    DT = 0
    returner = []
    dsdg = -0.033574
    dsdt = -0.00023335
    dsdm = 0.0104
    dfdg = -0.5227
    dfdt = 0.001455
    dfdm = -0.3574 
    dydg = -0.13522
    dydt = 0.00041
    dydm = -0.8345
    A = np.array([[dsdg, dsdt, dsdm],
         [dfdg, dfdt, dfdm],
         [dydg, dydt, dydm]])
    A_inv = np.linalg.inv(A)
    nlte_corrF=0#.2
    nlte_corrS=0#-0.027
    nlte_corrM=0#.1
    f=-1*((temp0[0])-(temp1[0]+nlte_corrF))
    s = -1*(temp0[1]+nlte_corrS)
    m = -1*(temp0[0]-(M+nlte_corrM))
    B = np.array([s, f, m])
    if abs(temp0[1]+nlte_corrS) > 0.0001 or abs((temp0[0])-(temp1[0]+nlte_corrF)) > 0.0003 or abs(M - (temp0[0])+nlte_corrM)>0.001:
        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
        DG = (np.dot(A_inv, B))[0]
        DT = (np.dot(A_inv, B))[1]
        DM = (np.dot(A_inv, B))[2]
        if abs(DG) > 0.5:
            DG=0.5*(DG/abs(DG))
        if abs(DT) > 250:
            DT = 250*(DT/abs(DT))
        if abs(DM) > 0.5:
            DM = 0.5*(DM/abs(DM))
        temperatu=temperatu+DT
        G=G+DG
        teff= temperatu
        #temp0[0] = temp0[0]+DM
        crea_ATMinterpol(teff, G, temp0[0]+nlte_corrM, nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{(temp0[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{(temp0[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{(temp0[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        g=G
        met = temp0[0]
    else:
        nome_file1 = nome_file
        teff = temperatu
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    return returner
def leggi_log(path_cartella_log):
    '''Di seguito i nomi delle colonne di log.txt. Restituisce la tavola letta con Table.read(). T      G      M      FeI            FeII           FeI-M/H        FeI-FeII       s(FeI) '''
    olddir = os.getcwd()
    os.chdir(path_cartella_log)
    log_table = Table.read("log.txt", format='ascii')
    return log_table
def minimo_tavola(tab, nome_colonna, toll=2.0, mode='abs'):
    '''Passata una tabella restituisce l'elemento minimo della colonna specificata e il relativo indice di riga. mode=abs trova il minimo in valore assoluto, mode=n trova il minimo considerando anche il segno'''
    x=tab[nome_colonna]
    if mode=='abs':
        X = abs(x)
    if mode=='n':
        X = x
    returner = []
    min = 0
    i=0
    absmin = X[X.argmin()]
    while min < absmin*toll or min-absmin<=0.05:
        M = X.argmin()
        min = X[M]
        min_index = M
        returner.append([min, int(min_index)])
        X[np.where(X == min)] = 1
        i=i+1
    return returner
def final_atmo(path_cartella_log): #
    '''Passa log file, restituisce la best atmosfera. Nelle cartelle contenenti le atmosfere analizzate è possibile creare un log file attraverso la funzione folderLog. Passa qua la directory del log di quella stella. Si cerca prima l'atmosfera che più minimizza FeI-FeII. Da qua si verifica che l'atmosfera che minimizza la slope sia sufficientemente vicina al minimo (i.e. entro 1.2*minimo_slope).'''
    tab=leggi_log(path_cartella_log)
    dF=minimo_tavola(tab, 'FeI-FeII')
    slope=minimo_tavola(tab, 's(FeI)')
    if (tab[slope[0][1]] == tab[dF[0][1]]) and (tab[dF[0][1]]['FeI-M/H'] <= 0.01):
        print(tab[dF[0][1]])
        print()
        return tab[dF[0][1]]
    elif esiste_in_lista(dF[0][1], slope):
        print(tab[dF[0][1]])
        print()
        return tab[dF[0][1]]
    elif esiste_in_lista(slope[0][1], dF):
        print(tab[slope[0][1]])
        print()
        return tab[slope[0][1]]
    elif esiste_in_lista(dF[1][1], slope): 
        print(tab[dF[1][1]])
        print()
        return tab[dF[1][1]]
    elif esiste_in_lista(slope[1][1], dF): 
        print(tab[slope[1][1]])
        print()
        return tab[slope[1][1]]
    elif len(dF)>= 3 and esiste_in_lista(dF[2][1], slope): 
        print(tab[dF[2][1]])
        print()
        return tab[dF[2][1]]
    elif len(slope)>= 3:
        if esiste_in_lista(slope[2][1], dF): 
            print(tab[slope[2][1]])
            print()
            return tab[slope[2][1]]
    else:
        print("nessuna atmosfera trovata.. :C")
def esiste_in_lista(x, lista2):
    esiste = any(x == sub2[1] for sub2 in lista2)
    return esiste
def NLTE_Fe_correction(t, e, g, x, wi):
    url = 'http://www.inspect-stars.com/cp/application.py/A_from_e?element_name=Fe'

    # Cambia il valore del campo 't'
    form_data = {
        't': f'{t}', 
        'e': f'{e}',
        'g': f'{g}',
        'x': f'{x}',
        'wi': f'{wi}'
    }

    # Codifica i dati come query string per l'invio con POST
    encoded_data = urllib.parse.urlencode(form_data).encode('utf-8')

    # Fai la richiesta POST con i dati del form
    req = urllib.request.Request(url, data=encoded_data)

    # Aggiungi un User-Agent per simulare un browser reale
    req.add_header('User-Agent', 'chrome')

    with urllib.request.urlopen(req) as response:
        result = response.read().decode('utf-8')

    # 1. Estrazione del blocco <pre>
    pre_block = re.search(r'<pre>(.*?)</pre>', result, re.DOTALL)
    if pre_block:
        pre_content = pre_block.group(1)  # Ottieni il contenuto tra <pre> e </pre>
        # 2. Estrazione dei numeri (inclusi decimali)
        numbers = re.findall(r'\d+\.\d+|\d+', pre_content)
        # Stampa o utilizza i numeri come preferisci
        print("Numeri estratti:", numbers)
        if len(numbers) == 5:
            return numbers
        else:
            returner = [e, 0, 0, 0, 0]
            return returner
    else:
        print("Blocco <pre> non trovato.")
def find_closest2(num, arr):
    arr = np.array(arr)
    closest_index = np.abs(arr - num).argmin()
    if (arr[closest_index]-num)>0.011:
        print("No line found!")
        return 0
    else:
        return arr[closest_index]
def createCorrectionGrid():
    o=0
    OLDDIR = os.getcwd()
    os.chdir(path_to_correctionGrids)
    with open('correction_grid.txt', 'w') as gridder:
        gridder.write("EW               T               G                V                deltaNLTE\n")
        for E in range(20, 200, 10):
            print(f"EW_riga: {E}")
            for i in range(4100, 5500, 50):
                for j in range(10, 26, 1):
                    for k in range(100, 200, 5):
                        #for l in range(120):
                        U=NLTE_Fe_correction(i, E, j/10, k/100, 34)
                        a0 = int(E-20)
                        a1 = int((i-4000)/20)
                        a2 = int((j-10))
                        a3 = int((k-100)/5)
                        gridder.write(f"{U[0]}               {i}               {j/10}                {k/100}                {U[3]}")
                        gridder.write("\n")
                        o=o+1
                        if o == 200:
                            time.sleep(np.random.exponential(10))
    os.chdir(OLDDIR)
"/home/starcat/Scrivania/ANALISIATM/ATMOSFERE/"
path_to_rnd = f"/home/starcat/Scrivania/ANALISIATM/rnd"
path_to_output = f"/home/starcat/Scrivania/ANALISIATM/ABBONDANZE_OUT/"
path_to_interpol = f"/home/starcat/Scrivania/ANALISIATM/interpol/PyKMOD-main/"
path_to_correctionGrids = f"/home/starcat/Scrivania/ANALISIATM/CorrectionGRID/"
####

N_stelle = 14
trials = 30
temp_log = np.zeros(trials)
logg_log = np.zeros(trials)
T_slope = np.zeros((trials, 2))
Logg_slope = np.zeros((trials, 2))
temp_best_guess1 = 0
temp_best_guess2 = 0
temp_best_guess3 = 0
temperatura = [3500, 3750, 4000, 4250, 4500, 4750, 5000, 5250, 5500, 5750, 6000, 6250, 6500, 6750, 7000, 7250, 7500, 7750, 8000, 8250, 8500, 8750, 9000, 9250, 9500, 9750, 10000, 10250, 10500, 10750, 11000, 11250, 11500, 11750, 12000, 12250, 12500, 12750, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000, 26000, 27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000, 50000]
logg_arr = [0, 0.50000, 1.00000, 1.50000, 2.00000, 2.50000, 3.00000, 3.50000, 4.00000, 4.50000, 5.00000]
MH = ["05", "02", "00", "-05", "-10", "-15", "-20"]
metallicity = ["0.5", "0.2", "0.0", "-0.5", "-1.0", "-1.5", "-2.0", "-2.5"]
metallic = [0.5, 0.2, 0.0, -0.5, -1.0, -1.5, -2.0, -2.5]
metallic_header = [-2.5, 2.0, -1.5, -1.0, -0.5, 0.0, 0.2, 0.5]
T_EFF_int = np.arange(3500, 5500 + 1, 25)
LOGG_int = np.arange(0.5, 5 + 0.06, 0.05)
MH_int = np.arange(0.0, -2.5 - 0.06, -0.05)
T_EFF_Nint = np.arange(3500, 6000 , 250)
LOGG_Nint = np.arange(0.5, 5 + 0.06, 0.5)
MH_Nint = np.arange(0.0, -2.5 - 0.06, -0.5)

ABUNDANCE_SCALE = [3.16228, 1.58489, 1.00000, 0.31620, 0.10000, 0.03162, 0.01000, 0.00316]
ABUNDANCE_SCALE_header = [0.00316, 0.01000, 0.03162, 0.10000, 0.31620, 1.00000, 1.58489, 3.16228]
minceData = [["HARPN1", 4746, 1.89, -0.56],
             ["HARPN2", 4610, 1.73, -0.58],
             ["HARPN3", 4458, 1.54, -0.56],
             ["HARPN4", 4045, 1.21, -0.64],
             ["HARPN5", 4444, 4,-0.5],
             ["HARPN6", 4722, 1.89, 0.02],
             ["HARPN7", 4725, 2.41, -0.19],
             ["HARPN8", 4444, 4, -0.5],
             ["HARPN9", 4458, 1.54, -0.56],
             ["HARPN10", 4393, 1.08, -1.99],
             ["HARPN11", 4193, 0.71, -1.95],
             ["HARPN12", 4167, 0.82, -1.45],
             ["HARPN13", 4154, 0.52, -2.09],
             ["HARPN14", 4287, 0.70, -1.71],
             ["HARPN15", 4316, 0.87, -1.46],
             ["HARPN16", 4276, 0.70, -2.27],
             ["HARPN17", 4249, 0.79, -1.61]]
NLTE_wavelenght = [3440.610, 
                   436.886,
            3440.990,
            3465.860,
            3475.440,
            3490.570,
            3565.380,
            3570.100,
            3581.190,
            3608.860,
            3618.770,
            3719.920,
            3737.120,
            3727.620,
            3745.560,
            3748.260,
            3758.230,
            3763.790,
            3815.840,
            3820.430,
            3824.440,
            3825.870,
            3840.440,
            3856.370,
            3859.910,
            3922.910,
            4045.810,
            4063.590,
            4071.740,
            4235.940,
            4250.120,
            4250.790,
            4415.120,
            4445.470,
            4494.560,
            4920.500,
            4994.130,
            5044.210,
            5198.700,
            5216.270,
            5225.520,
            5232.940,
            5236.200,
            5242.490,
            5247.040,
            5250.210,
            5269.540,
            5281.790,
            5379.570,
            5383.370,
            5434.521,
            5491.830,
            5586.750,
            5600.220,
            5661.350,
            5662.520,
            5696.090,
            5701.540,
            5705.460,
            5778.450,
            5784.660,
            5855.080,
            5916.250,
            5956.690,
            6065.479,
            6082.710,
            6151.620,
            6173.330,
            6200.310,
            6219.280,
            6240.650,
            6252.560,
            6265.130,
            6297.790,
            6311.500,
            6430.850,
            6498.940,
            6518.370,
            6574.229,
            6593.870,
            6609.110,
            6699.140,
            6726.670,
            6739.521,
            6750.150,
            6793.260,
            6810.260,
            6837.010,
            6854.820,
            6945.200,
            6978.850,
            7401.680,
            7912.870,
            8293.500,
            4233.170,
            4491.390,
            4508.290,
            4576.330,
            4582.840,
            4583.830,
            4620.520,
            4923.930,
            5018.440,
            5169.020,
            5197.580,
            5234.620,
            5264.810,
            5284.110,
            5325.550,
            5414.070,
            5425.260,
            6239.950,
            6247.560,
            6369.460,
            6432.680,
            6456.380,
            6516.080,
            7222.390,
            7224.490,
            7515.830,
            7711.720]

def stampaOra():
    current_time = time.localtime()
    formatted_time = time.strftime("%H:%M:%S", current_time)
    print("L'ora corrente è:", formatted_time)
def correzione(M):
    #return -0.15-0.075*M
    return 0
def findBestAtmosphereinFolder(path, mode, attiva_tolleranza=True, tolleranzaG=0.2, tolleranzaT=0.2, elemento1=26.0, elemento2=26.1, requiredT=0, requiredG=0, excludeT=0, excludeG=0): #tolleranza permette di tener di conto di un'eventuale discrepanza tra l'abbondanza di FeI e FeII (una temperatura può essere più accurata anche se la differenza tra abbondanza di FeI e FeII è maggiore di quella ottenuta per un altro logg e temperatura). Non attivare se si ha solo 1 elemento
    """ output:
     mode T: elem1, slope, std, temp, logg, elem2
     mode G: FeI, FeII, G, M
     mode A: FeI FeII T G M s(FeI) stdFeI stdFeII"""
    old_dir = os.getcwd()
    os.chdir(path)
    a = os.listdir()
    b = []
    os.chdir(old_dir)
    if mode=="T":
        if requiredG==0:
            returner = np.zeros(6)
            for i in range(len(a)):
                if a[i][0]=='T':
                    b.append(a[i])
            H = np.zeros((len(b), 7)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
            min = 10
            j=0
            for i in range(len(b)):
                tempS = simpleAnalysisSingle(path, b[i], 26.0)
                tempD = simpleAnalysisDouble(path, b[i], elemento1, elemento2)
                H[i][0] = tempS[1]
                H[i][1] = float(tempD[0])
                H[i][2] = float(tempD[1])
                H[i][3] = (parseFileName(path, b[i]))[1]
                H[i][4] = (parseFileName(path, b[i]))[2]
                H[i][5] = (parseFileName(path, b[i]))[0]
                H[i][6] = tempS[2]
                if attiva_tolleranza:
                    if (abs(H[i][2]-H[i][1]) < tolleranzaG) and (abs(H[i][0]) < abs(min)):
                        min = H[i][0]
                        j=i
                else:
                    if abs(H[i][2]-H[i][1]) < tolleranzaG and (abs(H[i][0]) < abs(min)):
                        min = H[i][0]
                        j=i
            returner[0]=H[j][1] #el1
            returner[1]=H[j][0] #slope
            returner[2]=H[j][6] #std
            returner[3]=H[j][3] #temp
            returner[4]=H[j][4] #logg
            returner[5]=H[j][2] #el2
            

            return returner
        else:
            returner = np.zeros(6)
            for i in range(len(a)):
                if a[i][0]=='T':
                    b.append(a[i])
            H = np.zeros((len(b), 7)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
            min = 10
            j=0
            for i in range(len(b)):
                tempS = simpleAnalysisSingle(path, b[i], 26.0)
                tempD = simpleAnalysisDouble(path, b[i], elemento1, elemento2)
                H[i][0] = tempS[1]
                H[i][1] = float(tempD[0])
                H[i][2] = float(tempD[1])
                H[i][3] = (parseFileName(path, b[i]))[1]
                H[i][4] = (parseFileName(path, b[i]))[2]
                H[i][5] = (parseFileName(path, b[i]))[0]
                H[i][6] = tempS[2]
                if attiva_tolleranza:
                    if abs(H[i][2]-H[i][1]) < tolleranzaG and (abs(H[i][0]) < abs(min)) and H[i][4]==requiredG:
                        min = H[i][0]
                        j=i
                else:
                    if (abs(H[i][0]) < abs(min)):
                        min = H[i][0]
                        j=i
            returner[0]=H[j][1] #el1
            returner[1]=H[j][0] #slope
            returner[2]=H[j][6] #std
            returner[3]=H[j][3] #temp
            returner[4]=H[j][4] #logg
            returner[5]=H[j][2] #el2
            

            return returner
        
    elif mode=="G":
        returner = np.zeros(4)
        for i in range(len(a)):
            if a[i][0]=='G':
                b.append(a[i])
            H = np.zeros((len(b), 7)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
        min = 10
        j=0
        for i in range(len(b)):
            tempS = simpleAnalysisSingle(path, b[i], 26.0)
            tempD = simpleAnalysisDouble(path, b[i], elemento1, elemento2)
            H[i][0] = tempS[1]
            H[i][1] = float(tempD[0])
            H[i][2] = float(tempD[1])
            H[i][3] = (parseFileName(path, b[i]))[1]
            H[i][4] = (parseFileName(path, b[i]))[2]
            H[i][5] = (parseFileName(path, b[i]))[0]
            H[i][6] = tempS[2]
            if attiva_tolleranza:
                if abs(H[i][2]-H[i][1]) < tolleranzaT and (abs(H[i][0]) < abs(min)):
                    min = H[i][0]
                    j=i
            else:
                if (abs(H[i][0]) < abs(min)):
                    min = H[i][0]
                    j=i
        returner[0]=H[j][1]
        returner[1]=H[j][2]
        returner[2]=H[j][4]
        returner[3]=H[j][5]
        return returner
    elif mode=="A":
        returner = np.zeros(8)
        for i in range(len(a)): #and (a[i][0]!='l' and a[i][0] != 'e'):
            b.append(a[i])
        H = np.zeros((len(b), 8)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
        min = 10
        j=0
        for i in range(len(b)):
            tempD = simpleAnalysisSingle(path, b[i], 26.1)
            tempS = simpleAnalysisSingle(path, b[i], 26.0)
            H[i][0] = tempS[1]
            H[i][1] = float(tempS[0])
            H[i][2] = float(tempD[0])
            H[i][3] = (sparseFileName(path, b[i]))[1]
            H[i][4] = (sparseFileName(path, b[i]))[2]
            H[i][5] = (sparseFileName(path, b[i]))[0]
            H[i][6] = tempS[2]
            H[i][7] = tempD[2]
            if attiva_tolleranza:
                if abs(H[i][2]-H[i][1]) < tolleranzaG and abs(H[i][0]) < tolleranzaT and abs(H[i][5]-H[i][1]) < 0.6 and H[i][6] < 0.2 and H[i][0] < min:
                    min = H[i][0]
                    j=i
            # else:
            #     if (abs(H[i][0]) < abs(min)):
            #         min = H[i][0]
            #         j=i
        returner[0]=H[j][1] 
        returner[1]=H[j][2] 
        returner[2]=H[j][3]
        returner[3]=H[j][4]
        returner[4]=H[j][5]
        returner[5]=H[j][0]
        returner[6]=H[j][6]
        returner[7]=H[j][7]
        return returner
    else:
        print("mode deve essere 'T' oppure 'G', per valutare le rispettivamente le abbondanze dedotte muovendosi su temperatura o logg")
        return None
def parseFileName(path, nome_file):
    olddir=os.getcwd()
    os.chdir(path)
    pattern = f"{nome_file[0]}"+r'STELLA(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    if match:
        returner.append(float(match.group(1)))
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        return returner
    else:
        return None
def sparseFileName(path, nome_file):
    olddir=os.getcwd()
    os.chdir(path)
    pattern = r'STELLA(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    if match:
        returner.append(float(match.group(1)))
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        return returner
    else:
        return None
def sparseFileName2(path, nome_file):
    olddir=os.getcwd()
    os.chdir(path)
    pattern = r'AH_STELLA(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    if match:
        returner.append(float(match.group(1)))
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        return returner
    else:
        return None
def make_set(T, G, M, DT=500, DG=1, DM=1):
    set1 = [[T-DT, G+DG, M-DM], [T, G+DG, M-DM], [T+DT, G+DG, M-DM], [T-DT, G, M-DM]]
    set2 = [[T, G, M-DM], [T+DT, G, M-DM], [T-DT, G-DG, M-DM], [T, G-DG, M-DM]]
    set3 = [[T+DT, G-DG, M-DM],[T-DT, G+DG, M], [T, G+DG, M], [T+DT, G+DG, M]]
    set4 = [[T-DT, G, M], [T, G, M], [T+DT, G, M], [T-DT, G-DG, M]]
    set5 = [[T, G-DG, M], [T+DT, G-DG, M], [T-DT, G+DG, M+DM], [T, G+DG, M+DM]]
    set6 = [[T+DT, G+DG, M+DM], [T-DT, G, M+DM], [T, G, M+DM], [T+DT, G, M+DM]]
    set7 = [[T-DT, G-DG, M+DM], [T, G-DG, M+DM], [T+DT, G-DG, M+DM]]
    SET_ALL = [set1, set2, set3, set4, set5, set6, set7]
    return SET_ALL
def searchPatch(nome_stella, T=4750, G=1.50, M=-1.00, DT=500, DG=1, DM=1):
    SET_ALL = make_set(T-250, G, M)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T-250, G+0.5, M)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T-250, G, M-0.5)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T-250, G+0.5, M-0.5)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T, G+0.5, M)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T, G, M-0.5)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T, G+0.5, M-0.5)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
def findAbundancePatch(atmoParameters, nome_stella, n_istanze=4, path_to_output=path_to_output, label='', multiple_elements=0,  elemento1=26.0, elemento2=26.1):
    """Restituisce: multiple element==0 media, slope, std, multiple element==1 media el1 e media el2"""
    preparaAbundance(nome_stella)
    bashcmd = f"mkdir {path_to_output}{nome_stella}"
    os.system(bashcmd)
    copia("stellaFes", path_to_abundance, path_to_abundance_0, "stellaFes")
    copia("stellaFes", path_to_abundance, path_to_abundance_1, "stellaFes")
    copia("stellaFes", path_to_abundance, path_to_abundance_2, "stellaFes")
    if n_istanze == 1:
        copia(f"atmoT{int(atmoParameters[0][0])}{atmoParameters[0][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[0][2]}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[0][0], atmoParameters[0][1], atmoParameters[0][2]))  #fai paramfile di abundance
        SlanciaAbundance(n_istanze)
        temp = path_to_output
        path_to_output = path_to_output + f"{nome_stella}"
        sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{atmoParameters[0][2]}_{int(atmoParameters[0][0])}_{atmoParameters[0][1]}") #copia l'output di abundance nella cartella degli output
        path_to_output=temp
    if n_istanze == 2:
        copia(f"atmoT{int(atmoParameters[0][0])}{atmoParameters[0][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[0][2]}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[0][0], atmoParameters[0][1], atmoParameters[0][2]), 0)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[1][0])}{atmoParameters[1][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[1][2]}", path_to_abundance_1, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[1][0], atmoParameters[1][1], atmoParameters[1][2]), 1)  #fai paramfile di abundance
        SlanciaAbundance(n_istanze)
        temp = path_to_output
        path_to_output = path_to_output + f"{nome_stella}"
        sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{atmoParameters[0][2]}_{int(atmoParameters[0][0])}_{atmoParameters[0][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS1", path_to_abundance_0, path_to_output, f"{label}STELLA{atmoParameters[1][2]}_{int(atmoParameters[1][0])}_{atmoParameters[1][1]}") #copia l'output di abundance nella cartella degli output
        path_to_output=temp
    if n_istanze == 3:
        copia(f"atmoT{int(atmoParameters[0][0])}{atmoParameters[0][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[0][2]}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[0][0], atmoParameters[0][1], atmoParameters[0][2]), 0)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[1][0])}{atmoParameters[1][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[1][2]}", path_to_abundance_0, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[1][0], atmoParameters[1][1], atmoParameters[1][2]), 1)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[2][0])}{atmoParameters[2][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[2][2]}", path_to_abundance_1, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[2][0], atmoParameters[2][1], atmoParameters[2][2]), 2)  #fai paramfile di abundance
        SlanciaAbundance(n_istanze)
        temp = path_to_output
        path_to_output = path_to_output + f"{nome_stella}"
        sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{atmoParameters[0][2]}_{int(atmoParameters[0][0])}_{atmoParameters[0][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS1", path_to_abundance_0, path_to_output, f"{label}STELLA{atmoParameters[1][2]}_{int(atmoParameters[1][0])}_{atmoParameters[1][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS2", path_to_abundance_1, path_to_output, f"{label}STELLA{atmoParameters[2][2]}_{int(atmoParameters[2][0])}_{atmoParameters[2][1]}") #copia l'output di abundance nella cartella degli output
        path_to_output=temp
    if n_istanze == 4:
        copia(f"atmoT{int(atmoParameters[0][0])}{atmoParameters[0][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[0][2]}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[0][0], atmoParameters[0][1], atmoParameters[0][2]), 0)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[1][0])}{atmoParameters[1][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[1][2]}", path_to_abundance_0, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[1][0], atmoParameters[1][1], atmoParameters[1][2]), 1)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[2][0])}{atmoParameters[2][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[2][2]}", path_to_abundance_1, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[2][0], atmoParameters[2][1], atmoParameters[2][2]), 2)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[3][0])}{atmoParameters[3][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[3][2]}", path_to_abundance_2, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[3][0], atmoParameters[3][1], atmoParameters[3][2]), 3)  #fai paramfile di abundance
        SlanciaAbundance(n_istanze)                                  #lancia abundance
        temp = path_to_output
        path_to_output = path_to_output + f"{nome_stella}"
        sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{atmoParameters[0][2]}_{int(atmoParameters[0][0])}_{atmoParameters[0][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS1", path_to_abundance_0, path_to_output, f"{label}STELLA{atmoParameters[1][2]}_{int(atmoParameters[1][0])}_{atmoParameters[1][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS2", path_to_abundance_1, path_to_output, f"{label}STELLA{atmoParameters[2][2]}_{int(atmoParameters[2][0])}_{atmoParameters[2][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS3", path_to_abundance_2, path_to_output, f"{label}STELLA{atmoParameters[3][2]}_{int(atmoParameters[3][0])}_{atmoParameters[3][1]}") #copia l'output di abundance nella cartella degli output
        path_to_output=temp
    # if multiple_elements == 0:
    #     results = simpleAnalysisSingle(path_to_output, f"TSTELLA{metal}_{int(tempera)}_{logG}", elemento1)
    # else:
    #     results = simpleAnalysisDouble(path_to_output, f"GSTELLA{metal}_{int(tempera)}_{logG}", elemento1, elemento2)
    # return results
def iterative_Search(tempera, logG, metal, trials=8, elemento=26.0, controller=1):
    diarioTemp = np.zeros((trials, 4))
    i=0
    min = 1000
    control = 1
    while controller==1 and i < trials:
        print(f"trial = {i}")
        file_check = f"{path_to_output}/TSTELLA{metal}_{int(tempera)}_{logG}"
        file_checkG = f"{path_to_output}/GSTELLA{metal}_{int(tempera)}_{logG}"
        if (exists(file_check)):
            break
        A = findAbundance(tempera, logG, metal, "T", 0, elemento)
        diarioTemp[i][0] = A[0]
        diarioTemp[i][1] = A[1]
        diarioTemp[i][2] = A[2]
        diarioTemp[i][3] = tempera
        if (i>=1) and (abs(diarioTemp[i][1])>abs(diarioTemp[i-1][1])):
            break
        metal = find_closest(A[0], metallic)
        required_index = 0
        for k in range(trials):
            if (k!=i) and (diarioTemp[k][3]==diarioTemp[i][3]):
                controller = 0
        if ((tempera > 3499) and (tempera < 12750)): 
            if (diarioTemp[i][1]>0):
                tempera = tempera + 250
                tempera = int(tempera)
            else:
                if tempera > 3749:
                    tempera = tempera - 250
                    tempera = int(tempera)
        i=i+1
    if controller == 1:
        print("il numero di trials non è stato sufficiente ad ottenere un risultato, prova con un numero di trials maggiore")
        if i != 0:
            for j in range(trials):
                if (diarioTemp[j][3] != 0) and (abs(diarioTemp[j][1]) < abs(min)): #!= 0 perché viene inizializzato con alcuni 0 che potrebbero non essere utilizzati, in tal caso ci si ferma prima. Si cerca la miglior slope, infatti si ricerca min e via via si sostituisce con la slope più bassa. Alla slope più bassa, la j-esima, corrisponde la j-esima atmosfera
                    min = diarioTemp[j][1]
                    required_index = j
            returner = np.zeros(4)
            returner[0] = diarioTemp[required_index][0] #media
            returner[1] = diarioTemp[required_index][1] #slope
            returner[2] = diarioTemp[required_index][2] #std
            returner[3] = diarioTemp[required_index][3] #temperatura
            return returner
    if controller == 0:
        for j in range(trials):
            if (diarioTemp[j][3] != 0) and (abs(diarioTemp[j][1]) < abs(min)): #!= 0 perché viene inizializzato con alcuni 0 che potrebbero non essere utilizzati, in tal caso ci si ferma prima. Si cerca la miglior slope, infatti si ricerca min e via via si sostituisce con la slope più bassa. Alla slope più bassa, la j-esima, corrisponde la j-esima atmosfera
                min = diarioTemp[j][1]
                required_index = j
        print(f"la temperatura {diarioTemp[required_index][3]} è il valore più vicino a quello cercato \ncorrispondente a una slope di {diarioTemp[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioTemp[required_index][0]} +- {diarioTemp[required_index][2]}")
        returner = np.zeros(4)
        returner[0] = diarioTemp[required_index][0] #media
        returner[1] = diarioTemp[required_index][1] #slope
        returner[2] = diarioTemp[required_index][2] #std
        returner[3] = diarioTemp[required_index][3] #temperatura
        return returner
def linear_func(x, a, b):
    return b +a*x
def expo_func(x, a, b):
    return a * np.exp(b*x)
def crea_ATMinterpol(T, logg, MH_g, nome_Stella, T0=0, logg0=0, MH_g0=0, n_istanze=1): #ritorna il nome del file prodotto da abundance con l'atmosfera creata tramite interpolazione
    '''T: temperatura (kelvin)
    logg: gravità superficiale (dex)
    MH_g: [M/H] (dex)'''
    os.chdir(path_to_interpol)
    if n_istanze == 1:
        vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg

        commandInterpol = f"/home/starcat/anaconda/bin/python pykmod.py {T} {logg} {vturb} {MH_g}"
        os.system(commandInterpol)

        #### FORMATTA KURUCZ ####
        header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
        header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
        tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
        def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
            with open(input_file, 'r') as f_in:
                lines = f_in.readlines()

            # Remove the first 3 rows and the last 2 rows
            lines = lines[3:-2]

            with open(output_file, 'w') as f_out:
                f_out.write(header1)
                f_out.write(header2)
                f_out.writelines(lines)
                f_out.write(tail)

        read_and_copy_file('modelatmosphere.txt', 'out.txt')
        comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
        os.system(comandoATMOS)
        preparaAbundance(nome_Stella)
        os.chdir(path_to_abundance)
        creaParamfileAbnd(vturbo(T, logg, MH_g))
        SlanciaAbundance()
        nome_output = f"{nome_Stella}_INT{MH_g:.2f}_{T:.0f}_{logg:.2f}"
        comando_bsh2 = f"mv PINOLOS {path_to_output}/{nome_output}"
        os.system(comando_bsh2)
        return nome_output
    if n_istanze == 2:
        ##### prima atmosfera
        vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg

        commandInterpol = f"/home/starcat/anaconda/bin/python pykmod.py {T} {logg} {vturb} {MH_g}"
        print(vturb)
        os.system(commandInterpol)

        #### FORMATTA KURUCZ ####
        header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
        header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
        tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
        def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
            with open(input_file, 'r') as f_in:
                lines = f_in.readlines()

            # Remove the first 3 rows and the last 2 rows
            lines = lines[3:-2]

            with open(output_file, 'w') as f_out:
                f_out.write(header1)
                f_out.write(header2)
                f_out.writelines(lines)
                f_out.write(tail)

        read_and_copy_file('modelatmosphere.txt', 'out.txt')
        comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
        os.system(comandoATMOS)
        preparaAbundance(nome_Stella)
        os.chdir(path_to_abundance)
        creaParamfileAbnd(vturbo(T, logg, MH_g))
        ##### seconda atmosfera
        vturb0 = 0.14 - 0.08 * (MH_g0) + 4.9 * T / 10000 - 0.47 * logg0

        commandInterpol = f"/home/starcat/anaconda/bin/python pykmod.py {T0} {logg0} {vturb0} {MH_g0}"
        print(vturb)
        os.system(commandInterpol)

        #### FORMATTA KURUCZ ####
        header1 = f"TEFF   {T0}.  GRAVITY {logg0} LTE\nTITLE  [{MH_g0}] VTURB={vturb0}  L/H=1.25 NOVER NEW ODF\n"
        header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g0):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
        tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
        def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
            with open(input_file, 'r') as f_in:
                lines = f_in.readlines()

            # Remove the first 3 rows and the last 2 rows
            lines = lines[3:-2]

            with open(output_file, 'w') as f_out:
                f_out.write(header1)
                f_out.write(header2)
                f_out.writelines(lines)
                f_out.write(tail)

        read_and_copy_file('modelatmosphere.txt', 'out.txt')
        comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
        os.system(comandoATMOS)
        preparaAbundance(nome_Stella)
        os.chdir(path_to_abundance)
        creaParamfileAbnd(vturbo(T, logg, MH_g))
        SlanciaAbundance(2)
        nome_output = f"{nome_Stella}_INT{MH_g:.2f}_{T:.0f}_{logg:.2f}"
        comando_bsh2 = f"mv PINOLOS {path_to_output}/{nome_output}"
        os.system(comando_bsh2)
        return lista_nome_output
def sposta(nome_oggetto, path_oggetto, path_destinazione, nome_oggetto_destinazione):
    in_dir = os.getcwd()
    if path_oggetto[-1] == "/":
        path_obj = path_oggetto[:-1]
    else:
        path_obj = path_oggetto 
    if path_destinazione[-1] == "/":
        path_D = path_destinazione[:-1]
    else:
        path_D = path_destinazione
    COMANDO = f"mv {path_obj}/{nome_oggetto} {path_D}/{nome_oggetto_destinazione}"
    os.system(COMANDO)
    os.chdir(in_dir)
def copia(nome_oggetto, path_oggetto, path_destinazione, nome_oggetto_destinazione):
    in_dir = os.getcwd()
    if path_oggetto[-1] == "/":
        path_obj = path_oggetto[:-1]
    else:
        path_obj = path_oggetto 
    if path_destinazione[-1] == "/":
        path_D = path_destinazione[:-1]
    else:
        path_D = path_destinazione
    COMANDO = f"cp {path_obj}/{nome_oggetto} {path_D}/{nome_oggetto_destinazione}"
    os.system(COMANDO)
    os.chdir(in_dir)
def getTail(T, L, M):
    file_path = path_to_atmosphere + f"MH{M:.1f}/atmoT{int(T)}{L:.1f}.txt"
    # Leggi l'ultima riga del file
    with open(file_path, 'r') as file:
        last_line = file.readlines()[-2]
    # Estrai i numeri dalla riga usando una regex
    numbers = re.findall(r'[-+]?\d*\.\d+|[-+]?\d+', last_line)
    # Converti i numeri estratti in float
    numbers = [float(num) for num in numbers]
    numbers[0]

    if numbers[1] == 0.0:
        grandezza = "E+00"
    if numbers[1] < 0:
        grandezza = f"E-0{int(abs(numbers[1]))}"
    if numbers[1] > 0:
        grandezza = f"E+0{int(numbers[1])}"

    returner = f"{numbers[0]}" + grandezza
    return returner
def vturbo(T, LG, Met):
    return 0.14 - 0.08 * (Met) + 4.9 * T / 10000 - 0.47 * LG
def creaParamfileAbnd(microturb, numero_abundance=0, path_to_abundance=path_to_abundance, file_linee="stellaFes"):
    if numero_abundance==0:
        with open('param.txt', 'w') as paramfile:
            paramfile.write(f'ATMname ModelE\nLINESname {file_linee}\nOUTname PINOLOS\nvturb {str(microturb)}')
            sposta('param.txt', os.getcwd(), path_to_abundance, 'param.txt')
    if numero_abundance==1:
        with open('param.txt', 'w') as paramfile:
            paramfile.write(f'ATMname ModelE\nLINESname {file_linee}\nOUTname PINOLOS1\nvturb {str(microturb)}')
            sposta('param.txt', os.getcwd(), path_to_abundance_0, 'param.txt')
    if numero_abundance==2:
        with open('param.txt', 'w') as paramfile:
            paramfile.write(f'ATMname ModelE\nLINESname {file_linee}\nOUTname PINOLOS2\nvturb {str(microturb)}')
            sposta('param.txt', os.getcwd(), path_to_abundance_1, 'param.txt')
    if numero_abundance==3:
        with open('param.txt', 'w') as paramfile:
            paramfile.write(f'ATMname ModelE\nLINESname {file_linee}\nOUTname PINOLOS3\nvturb {str(microturb)}')
            sposta('param.txt', os.getcwd(), path_to_abundance_2, 'param.txt')
def SlanciaAbundance(n_istanze=1):
    if (n_istanze >= 1) and (n_istanze <= 4): 
        if n_istanze==1:
            DIR_INI = os.getcwd()
            os.chdir(path_to_abundance)
            abnd_path = "./abundance"
            process1 = subprocess.Popen([abnd_path, '-t'])
            process1.wait()
            os.chdir(DIR_INI)
        if n_istanze==2:
            DIR_INI = os.getcwd()
            os.chdir(path_to_abundance)
            abnd_path = "./abundance"
            process1 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_0)
            process2 = subprocess.Popen([abnd_path, '-t'])
            # Attendi che entrambi i processi terminino
            process1.wait()
            process2.wait()
            os.chdir(DIR_INI)
        if n_istanze==3:
            DIR_INI = os.getcwd()
            abnd_path = "./abundance"
            os.chdir(path_to_abundance)
            process1 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_0)
            process2 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_1)
            process3 = subprocess.Popen([abnd_path, '-t'])
            # Attendi che entrambi i processi terminino
            process1.wait()
            process2.wait()
            process3.wait()
            os.chdir(DIR_INI)
        if n_istanze==4:
            DIR_INI = os.getcwd()
            abnd_path = "./abundance"
            os.chdir(path_to_abundance)
            process1 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_0)
            process2 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_1)
            process3 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_2)
            process4 = subprocess.Popen([abnd_path, '-t'])
            # Attendi che entrambi i processi terminino
            process1.wait()
            process2.wait()
            process3.wait()
            process3.wait()
            os.chdir(DIR_INI)
    else:
        print(f"Numero di istanze assegnate DEVE essere compreso tra 1 e 4 inclusi. Inserito {n_istanze}")
        return None
def indici_elementiuguali(lista1, lista2):
    indici = [i for i, (x, y) in enumerate(zip(lista1, lista2)) if x == y]
    return indici
def folderLog(path, mode=1):
    '''Restituisce una lista con i parametri delle atmosfere vincenti'''
    old_dir = os.getcwd()
    os.chdir(path)
    a = os.listdir()
    b = []
    returner = np.zeros(8)
    for i in range(len(a)):
        if a[i] != "log.txt" and a[i] != "exit":
            b.append(a[i]) #in b i nomi delle stelle
    H = np.zeros((len(b), 8)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
    min = 10
    j=0
    with open("log.txt", 'w') as f:
        f.write("    T      G      M      FeI            FeII           FeI-M/H        FeI-FeII       s(FeI)    \n")
        for i in range(len(b)):
            tempD = simpleAnalysisSingle(path, b[i], 26.1)
            tempS = simpleAnalysisSingle(path, b[i], 26.0)
            H[i][0] = tempS[1]
            H[i][1] = float(tempS[0])
            H[i][2] = float(tempD[0])
            if mode == 1:
                H[i][3] = (sparseFileName(path, b[i]))[1]
                H[i][4] = (sparseFileName(path, b[i]))[2]
                H[i][5] = (sparseFileName(path, b[i]))[0]
            if mode == 2:
                H[i][3] = (sparseFileName2(path, b[i]))[1]
                H[i][4] = (sparseFileName2(path, b[i]))[2]
                H[i][5] = (sparseFileName2(path, b[i]))[0]
            H[i][6] = tempS[2]
            H[i][7] = tempD[2]
            f.write(f"{H[i][3]:.0f}   {H[i][4]:.2f}   {H[i][5]:.2f}   {H[i][1]:.3}+-{H[i][6]:.3f}   {H[i][2]:.3}+-{H[i][7]:.3f}   {H[i][1]-H[i][5]:.4f}         {H[i][1]-H[i][2]:.3f}          {H[i][0]:.4f}\n")
    winners = []
    for i in range(len(b)): #erano 0.6                            0.21                    0.11
        if (abs(H[i][1]-H[i][5]) < 0.17) and (abs(H[i][1]-H[i][2])<0.01) and (abs(H[i][0])<0.005):
            winners.append([H[i][3], H[i][4], H[i][6], 100])
    ds=0.002
    dg=0.04
    dm=0.08
    q=1
    for u in range(100):
        for i in range(len(b)):
            if len(winners) < 4:
                if (abs(H[i][1]-H[i][5])<0.1+dm) and (abs(H[i][1]-H[i][2])<0.01+dg) and (abs(H[i][0])<0.005+ds):
                    if 100-5*q <= -700:
                        winners.append([H[i][3], H[i][4], H[i][5], 10/q])
                    else:
                        winners.append([H[i][3], H[i][4], H[i][5], 100/q])#-5*(q)])
            q=q+1
            ds=ds+0.001
            dg=dg+0.005
            dm=dm+0.01
    os.chdir(old_dir)
    return winners
def tra_le_atmosfere(path_to_output=path_to_output, nome_stella='HARPN'):
    '''la miglior atmosfera, si restituisce una lista contenente le '''
    a=[]
    # findBestAtmosphereinFolder(path_to_output+f"HARPN{I}.fits", "A")
    for I in range(1, N_stelle+1):
        a.append(folderLog(path_to_output+f"{nome_stella}{I-1}.fits"))
    return a
def intermediateATM(atmospheric_param):
    RET = []
    return_param = np.zeros(3)
    if len(atmospheric_param) == 0:
        print("nessuna atmosfera passata!")
        return None
    elif len(atmospheric_param) == 1:
        return atmospheric_param[0]
    elif len(atmospheric_param) == 2:
        for i in range(3):
            return_param[i] = (1.0*atmospheric_param[0][i]+1.0*atmospheric_param[1][i])/2
        RET.append(return_param[0])
        RET.append(return_param[1])
        RET.append(return_param[2])
        return RET
    elif len(atmospheric_param) == 3:
        A = []
        A.append((atmospheric_param[0][0]+atmospheric_param[1][0]+atmospheric_param[2][0])/3)
        A.append((atmospheric_param[0][1]+atmospheric_param[1][1]+atmospheric_param[2][1])/3)
        A.append((atmospheric_param[0][2]+atmospheric_param[1][2]+atmospheric_param[2][2])/3)
        return A
    elif len(atmospheric_param) == 4:
        A = []
        normaliz = (atmospheric_param[0][3]+atmospheric_param[1][3]+atmospheric_param[2][3]+atmospheric_param[3][3]) #media ponderata sui pesi, a seconda di quanto è buona l'atmosfera
        A.append((atmospheric_param[0][0]*atmospheric_param[0][3]+atmospheric_param[1][0]*atmospheric_param[1][3]+atmospheric_param[2][0]*atmospheric_param[2][3]+atmospheric_param[3][0]*atmospheric_param[3][3])/normaliz)
        A.append((atmospheric_param[0][1]*atmospheric_param[0][3]+atmospheric_param[1][1]*atmospheric_param[1][3]+atmospheric_param[2][1]*atmospheric_param[2][3]+atmospheric_param[3][1]*atmospheric_param[3][3])/normaliz)
        A.append((atmospheric_param[0][2]*atmospheric_param[0][3]+atmospheric_param[1][2]*atmospheric_param[1][3]+atmospheric_param[2][2]*atmospheric_param[2][3]+atmospheric_param[3][2]*atmospheric_param[3][3])/normaliz)
        return A
    elif len(atmospheric_param) == 5:
        A = []
        normaliz = (atmospheric_param[0][3]+atmospheric_param[1][3]+atmospheric_param[2][3]+atmospheric_param[3][3]+atmospheric_param[4][3]) #media ponderata sui pesi, a seconda di quanto è buona l'atmosfera
        A.append((atmospheric_param[0][0]*atmospheric_param[0][3]+atmospheric_param[1][0]*atmospheric_param[1][3]+atmospheric_param[2][0]*atmospheric_param[2][3]+atmospheric_param[3][0]*atmospheric_param[3][3]+atmospheric_param[4][0]*atmospheric_param[4][3])/normaliz)
        A.append((atmospheric_param[0][1]*atmospheric_param[0][3]+atmospheric_param[1][1]*atmospheric_param[1][3]+atmospheric_param[2][1]*atmospheric_param[2][3]+atmospheric_param[3][1]*atmospheric_param[3][3]+atmospheric_param[4][1]*atmospheric_param[4][3])/normaliz)
        A.append((atmospheric_param[0][2]*atmospheric_param[0][3]+atmospheric_param[1][2]*atmospheric_param[1][3]+atmospheric_param[2][2]*atmospheric_param[2][3]+atmospheric_param[3][2]*atmospheric_param[3][3]+atmospheric_param[4][2]*atmospheric_param[4][3])/normaliz)
        return A
    else:
        A = [4750, 1.5, -1.0]
def stampaAnalisi(path_file, nome):
    A = simpleAnalysisSingle(path_file, nome, 26.0)
    B = simpleAnalysisSingle(path_file, nome, 26.1)
    with open("exit", 'w') as f:
        f.write(f"FeI = {A[0]}+{A[2]}\nFeII = {B[0]}+{B[2]}\n s(FeI) = {A[1]}\nDF = {A[0]-B[0]}")
    sposta("exit", os.getcwd(), path_file, "exit")
def simpleAnalysisSingle(path_file, nome_file, elemento=26.0): #passa path e nome file, restituisce abbondanza di un elemento, slope della retta che interpola i punti sul grafico abbondanza-energia potenziale, restituisce anche la standard deviation
    """Restituisce: media, slope, std"""
    DIR_INI = os.getcwd()
    os.chdir(path_file)
    returner = np.zeros(3)
    #t1 = Table.read('stellaFes', format='ascii')
    print(os.getcwd())
    t2 = Table.read(nome_file, format='ascii')
    x=t2['col3']
    y=t2['col7']
    fe=t2['col2']
    limit=((fe==elemento))
    x=x[limit]
    y=y[limit]
    limit=((y<np.mean(y)+1.5*np.std(y))&(y>np.mean(y)-1.5*np.std(y)))
    x=x[limit]
    y=y[limit]
    popt, pcov = curve_fit(linear_func, x, y,method='lm')
    std1=np.std(y-(linear_func(x,*popt)))
    #print(f"deviazione standard = {std1}")
    os.chdir(DIR_INI)
    returner[0] = np.mean(y) #media
    returner[1] = popt[0]    #slope
    returner[2] = std1       #std
    print(nome_file)
    return returner
def simpleAnalysisDouble(path_file, nome_file, elemento1=26.0, elemento2=26.1): #passa path del file, il nome, e restituisce le abbondanze di due elementi (o diverse ionizzazioni)
    """Restituisce: abbondanza el1, abbondanza el2"""
    DIR_INI = os.getcwd()
    os.chdir(path_file)
    returner = np.zeros(2)
    #t1 = Table.read('stellaFes', format='ascii')
    t2 = Table.read(nome_file, format='ascii')
    t3 = Table.read(nome_file, format='ascii')
    x2=t2['col3']
    y2=t2['col7']
    fe2=t2['col2']
    limit=((fe2==elemento1))
    x2=x2[limit]
    y2=y2[limit]
    limit=((y2<np.mean(y2)+1*np.std(y2))&(y2>np.mean(y2)-1*np.std(y2)))
    x2=x2[limit]
    y2=y2[limit]
    x3=t3['col3']
    y3=t3['col7']
    fe3=t3['col2']
    limit=((fe3==elemento2))
    x3=x3[limit]
    y3=y3[limit]
    limit=((y3<np.mean(y3)+1*np.std(y3))&(y3>np.mean(y3)-1*np.std(y3)))
    x3=x3[limit]
    y3=y3[limit]
    os.chdir(DIR_INI)
    returner[0] = np.mean(y2) #abbondanza elemento1
    returner[1] = np.mean(y3)+correzione(returner[0]) #abbondanza elemento2
    return returner
def findAbundance(tempera, logG, metal, label, multiple_elements=0,  elemento1=26.0, elemento2=26.1):
    """Restituisce: multiple element==0 media, slope, std, multiple element==1 media el1 e media el2"""
    copia(f"atmoT{int(tempera)}{logG}.txt", path_to_atmosphere + f"MH{metal}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
    creaParamfileAbnd(vturbo(tempera, logG, metal))  #fai paramfile di abundance
    SlanciaAbundance()                                    #lancia abundance
    sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{metal}_{int(tempera)}_{logG}") #copia l'output di abundance nella cartella degli output
    if multiple_elements == 0:
        results = simpleAnalysisSingle(path_to_output, f"TSTELLA{metal}_{int(tempera)}_{logG}", elemento1)
    else:
        results = simpleAnalysisDouble(path_to_output, f"GSTELLA{metal}_{int(tempera)}_{logG}", elemento1, elemento2)
    return results
def iterative_temperatureSearch(tempera, logG, metal, trials=8, elemento=26.0, controller=1):
    diarioTemp = np.zeros((trials, 4))
    i=0
    min = 1000
    control = 1
    while controller==1 and i < trials:
        print(f"trial = {i}")
        file_check = f"{path_to_output}/TSTELLA{metal}_{int(tempera)}_{logG}"
        file_checkG = f"{path_to_output}/GSTELLA{metal}_{int(tempera)}_{logG}"
        if (exists(file_check)):
            break
        # if (exists(file_checkG)) and control==1:
        #     if simpleAnalysisSingle(os.getcwd(), file_checkG)[1] > 0:
        #         tempera=tempera+250
        #     if simpleAnalysisSingle(os.getcwd(), file_checkG)[1] < 0:
        #         tempera=tempera-250
        #     control = 0
        A = findAbundance(tempera, logG, metal, "T", 0, elemento)
        diarioTemp[i][0] = A[0]
        diarioTemp[i][1] = A[1]
        diarioTemp[i][2] = A[2]
        diarioTemp[i][3] = tempera
        if (i>=1) and (abs(diarioTemp[i][1])>abs(diarioTemp[i-1][1])):
            break
        metal = find_closest(A[0], metallic)
        required_index = 0
        for k in range(trials):
            if (k!=i) and (diarioTemp[k][3]==diarioTemp[i][3]):
                controller = 0
        if ((tempera > 3499) and (tempera < 12750)): 
            if (diarioTemp[i][1]>0):
                tempera = tempera + 250
                tempera = int(tempera)
            else:
                if tempera > 3749:
                    tempera = tempera - 250
                    tempera = int(tempera)
        i=i+1
    if controller == 1:
        print("il numero di trials non è stato sufficiente ad ottenere un risultato, prova con un numero di trials maggiore")
        if i != 0:
            for j in range(trials):
                if (diarioTemp[j][3] != 0) and (abs(diarioTemp[j][1]) < abs(min)): #!= 0 perché viene inizializzato con alcuni 0 che potrebbero non essere utilizzati, in tal caso ci si ferma prima. Si cerca la miglior slope, infatti si ricerca min e via via si sostituisce con la slope più bassa. Alla slope più bassa, la j-esima, corrisponde la j-esima atmosfera
                    min = diarioTemp[j][1]
                    required_index = j
            #print(f"la temperatura {diarioTemp[required_index][3]} è il valore più vicino a quello cercato \ncorrispondente a una slope di {diarioTemp[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioTemp[required_index][0]} +- {diarioTemp[required_index][2]}")
            returner = np.zeros(4)
            returner[0] = diarioTemp[required_index][0] #media
            returner[1] = diarioTemp[required_index][1] #slope
            returner[2] = diarioTemp[required_index][2] #std
            returner[3] = diarioTemp[required_index][3] #temperatura
            return returner
    if controller == 0:
        for j in range(trials):
            if (diarioTemp[j][3] != 0) and (abs(diarioTemp[j][1]) < abs(min)): #!= 0 perché viene inizializzato con alcuni 0 che potrebbero non essere utilizzati, in tal caso ci si ferma prima. Si cerca la miglior slope, infatti si ricerca min e via via si sostituisce con la slope più bassa. Alla slope più bassa, la j-esima, corrisponde la j-esima atmosfera
                min = diarioTemp[j][1]
                required_index = j
        print(f"la temperatura {diarioTemp[required_index][3]} è il valore più vicino a quello cercato \ncorrispondente a una slope di {diarioTemp[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioTemp[required_index][0]} +- {diarioTemp[required_index][2]}")
        returner = np.zeros(4)
        returner[0] = diarioTemp[required_index][0] #media
        returner[1] = diarioTemp[required_index][1] #slope
        returner[2] = diarioTemp[required_index][2] #std
        returner[3] = diarioTemp[required_index][3] #temperatura
        return returner
def iterative_logGSearch(tempera, logG, metal, trials=8, elemento=26.0, controller=1):
    diarioLogG = np.zeros((trials, 3))
    diarioLogG = diarioLogG + 555.9
    i=0
    min = 1000
    while controller==1 and i<trials:
        print(f"trial = {i}")
        file_check = f"{path_to_output}/GSTELLA{metal}_{int(tempera)}_{logG}"
        file_checkT = f"{path_to_output}/TSTELLA{metal}_{int(tempera)}_{logG}"
        if (exists(file_check)):
            break
        # if (exists(file_checkT)):
        #     if simpleAnalysisDouble(os.getcwd(), file_checkT)[0]-simpleAnalysisDouble(os.getcwd(), file_checkT)[1] > 0:
        #         logG=logG+0.5
        #     if simpleAnalysisDouble(os.getcwd(), file_checkT)[0]-simpleAnalysisDouble(os.getcwd(), file_checkT)[1] < 0:
        #         logG=logG-0.5
        outPut = findAbundance(tempera, logG, metal, "G", 1, 26.0, 26.1)
        diarioLogG[i][0] = outPut[0]
        diarioLogG[i][1] = outPut[1]
        diarioLogG[i][2] = logG
        required_index = 0
        for k in range(trials):
            if (k!=i) and (diarioLogG[k][2]==diarioLogG[i][2]):
                controller = 0
        if ((logG > -0.1) and (logG < 5.1)): 
            if (diarioLogG[i][0] - diarioLogG[i][1] > 0):
                logG = logG + 0.5
            else:
                if logG > 0.4:
                    logG = logG - 0.5
        print(f"{diarioLogG[i][0]-diarioLogG[i][1]} con {diarioLogG[i][2]}")
        i=i+1
    if controller == 1:
        if i != 0:
            for j in range(i):
                if abs(diarioLogG[j][1]-diarioLogG[j][0]) < abs(min):
                    min = (diarioLogG[j][0]-diarioLogG[j][1])
                    required_index = j
            print(f"il logG {diarioLogG[required_index][2]} è il valore più vicino a quello cercato \ncorrispondente a elemento 26.1 {diarioLogG[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioLogG[required_index][0]}")
            returner = np.zeros(3)
            returner[0] = diarioLogG[required_index][0] #abbondanza elemento 1
            returner[1] = diarioLogG[required_index][1] #abbondanza elemento 2
            returner[2] = diarioLogG[required_index][2] #logG cercata
            return returner
    if controller == 0:
        for j in range(i):
            if abs(diarioLogG[j][1]-diarioLogG[j][0]) < abs(min):
                min = (diarioLogG[j][0]-diarioLogG[j][1])
                required_index = j
        print(f"il logG {diarioLogG[required_index][2]} è il valore più vicino a quello cercato \ncorrispondente a elemento 26.1 {diarioLogG[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioLogG[required_index][0]}")
        returner = np.zeros(3)
        returner[0] = diarioLogG[required_index][0] #abbondanza elemento 1
        returner[1] = diarioLogG[required_index][1] #abbondanza elemento 2
        returner[2] = diarioLogG[required_index][2] #logG cercata
        return returner                                 #Iron_lines#al posto di molto_linee per il ferro
def creaOptFileARES(specfits='Stello.fits', readlinedat='molte_linee', fileout='stella.ares', lambdai="4600.", lambdaf="6700.", smoothder="4", space="3.0", rejt="1;5764,5766,6047,6052,6068,6076", lineresol="0.1", miniline="2", plots_flag="0", rvmask="3,6021.8,6024.06,6027.06,6024.06,20"):
    with open('mine.opt', 'w') as paramfile:
        paramfile.write(f'specfits=\'{specfits}\'\nreadlinedat=\'{readlinedat}\'\nfileout=\'{fileout}\'\nlambdai={lambdai}\nlambdaf={lambdaf}\nsmoothder={smoothder}\nspace={space}\nrejt={rejt}\nlineresol={lineresol}\nminiline={miniline}\nplots_flag={plots_flag}\nrvmask=\'{rvmask}\'')
def parameterAnalysis(param_space_directions, T_center, log_center, M_center, T_steps=50, log_steps=0.1, M_steps=0.1, T_tot=8, log_tot=8, M_tot=10):
    if (param_space_directions < 1) or (param_space_directions>4):
        print("Il primo argomento in parameterAnalysis deve essere un int \n1 - muoviti solo in temperatura\n2 - muoviti in temperatura e logg\n3 - muoviti in temperatura logg e metallicità\n4 muoviti solo in logg")
    elif param_space_directions == 1:
        T = T_center - (T_steps*T_tot)/2
        T_temp = T
        ### muovi temperatura
        for i in range(T_tot):
            crea_ATMinterpol(round(T_temp), round(log_center, 1), round(M_center, 2))                
            T_temp = T_temp + T_steps
    elif param_space_directions == 2:
        T = T_center - (T_steps*T_tot)/2
        log = log_center - (log_steps*log_tot)/2
        T_temp = T
        log_temp = log
        ### muovi temperatura e al suo interno il logg
        for i in range(T_tot):
            for j in range(log_tot):
                crea_ATMinterpol(round(T_temp), round(log_temp, 1), round(M_center, 2))
                log_temp = log_temp + log_steps
            T_temp = T_temp + T_steps
            log_temp = log
    elif param_space_directions == 3:
        T = T_center - (T_steps*T_tot)/2
        log = log_center - (log_steps*log_tot)/2
        M = M_center - (M_steps*T_tot)/2
        T_temp = T
        log_temp = log
        M_temp = M
        ### muovi temperatura e al suo interno il logg
        for i in range(T_tot):
            for j in range(log_tot):
                for k in range(M_tot):
                    crea_ATMinterpol(round(T_temp), round(log_temp, 1), round(M_temp, 2))
                    M_temp = M_temp + M_steps
                log_temp = log_temp + log_steps
                M_temp = M
            T_temp = T_temp + T_steps
            log_temp = log
    print("FINITO")
def find_closest(num, arr):
    arr = np.array(arr)
    closest_index = np.abs(arr - num).argmin()
    return arr[closest_index]
def write_file(output_file, header1, header2, lines, tail):
        with open(output_file, 'w') as f_out:
            f_out.write(header1)
            f_out.write(header2)
            f_out.writelines(lines)
            f_out.write(tail)
def rimuoviHeader_atm(input_file, input_path, output_file, output_path):
    A=(os.getcwd())
    os.chdir(input_path)
    with open(input_file, 'r') as filein:
        lines = filein.readlines()
    lines = lines[23:-2]
    os.chdir(output_path)
    with open(output_file, 'w') as fileout:
        fileout.writelines(lines)
    os.chdir(A)
def SinterpolaT(T, logg, M, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    t_pradk = find_closest(T, T_EFF_Nint)
    T=find_closest(T, T_EFF_int) #temperatura più fine (25K) rispetto a logg e M qua sotto (che ancora sono su griglia vanilla)
    logg=find_closest(logg, LOGG_Nint)
    M=find_closest(M, MH_Nint)
    vturbs = vturbo(T, logg, M)
    header1 = f"TEFF   {T}.  GRAVITY {logg:.1f} LTE\nTITLE  [{M:.1f}] VTURB={vturbs}  L/H=1.25 NOVER NEW ODF\n"
    header2 = " OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   0.10000 ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    os.chdir(path_to_atmosphere + f"MH{M:.1f}")
    tail = f"PRADK {getTail(t_pradk, logg, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    for i in range(len(T_EFF_Nint)):
        nomeATMint = f'T{T_EFF_Nint[i]:.0f}{logg:.1f}.int'
        copia(f'atmoT{T_EFF_Nint[i]}{logg:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd())
        P.append(Table.read(f'T{T_EFF_Nint[i]:.0f}{logg:.1f}.int', format='ascii'))
    y = np.zeros((72, 10))
    for i in range(72):
        for j in range(10):
            a=[]
            for k in range(len(T_EFF_Nint)):
                a.append((P[k])[i][j])
            z = np.polyfit(T_EFF_Nint, np.asarray(a), 8)
            f = np.poly1d(z)
            x_new = np.linspace(T_EFF_int[0], T_EFF_int[-1], len(T_EFF_Nint)*1000)
            y_new = f(x_new)
            T = find_closest(T, x_new)
            indice = np.where(x_new == T)[0]
            y[i][j] = y_new[indice]
            if y[i][j]<0:
                y[i][j] = 0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'atm{T:.0f}_{logg}.int'
    write_file( nome_out, header1, header2, linea, tail)
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return P
def interpolaT(T, logg, M, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    t_pradk = find_closest(T, T_EFF_Nint)
    T=find_closest(T, T_EFF_int) #temperatura più fine (25K) rispetto a logg e M qua sotto (che ancora sono su griglia vanilla)
    logg=find_closest(logg, LOGG_Nint)
    M=find_closest(M, MH_Nint)
    vturbs = vturbo(T, logg, M)
    header1 = f"TEFF   {T}.  GRAVITY {logg:.1f} LTE\nTITLE  [{M:.1f}] VTURB={vturbs}  L/H=1.25 NOVER NEW ODF\n"
    header2 = " OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   0.10000 ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    os.chdir(path_to_atmosphere + f"MH{M:.1f}")
    tail = f"PRADK {getTail(t_pradk, logg, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    for i in range(len(T_EFF_Nint)):
        nomeATMint = f'T{T_EFF_Nint[i]:.0f}{logg:.1f}.int'
        copia(f'atmoT{T_EFF_Nint[i]}{logg:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd())
        P.append(Table.read(f'T{T_EFF_Nint[i]:.0f}{logg:.1f}.int', format='ascii'))
    y = np.zeros((72, 10))
    o=-1
    for i in range(72):
        for j in range(10):
            A=[]
            if (find_closest(t_pradk, T_EFF_Nint) == 3500):
                A.append((P[0])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[1])[i][j])
                A.append((P[2])[i][j])
                z = np.polyfit([T_EFF_Nint[0],T_EFF_Nint[1]], np.asarray(A), 2)         
            elif (find_closest(t_pradk, T_EFF_Nint) == 6000):
                A.append((P[-1])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[-2])[i][j])
                A.append((P[-3])[i][j])
                z = np.polyfit([T_EFF_Nint[-1],T_EFF_Nint[-2]], np.asarray(A), 2)  
            elif (find_closest(t_pradk, T_EFF_Nint) > 3500) and (find_closest(t_pradk, T_EFF_Nint) < 6000):
                for k in range(len(T_EFF_Nint)): #scorro su atmosfere
                    if (T > T_EFF_Nint[k-1]) and (T <= T_EFF_Nint[k]):
                        A.append((P[k-1])[i][j])
                        A.append((P[k])[i][j])
                        A.append((P[k+1])[i][j])
                        z = np.polyfit([T_EFF_Nint[k-1],T_EFF_Nint[k],T_EFF_Nint[k+1]], np.asarray(A), 2)
                        m=(A[1]-A[0])/(T_EFF_Nint[k]-T_EFF_Nint[k-1])
                        o=k-1
            f = np.poly1d(z)
            
            x_new = np.linspace(T_EFF_int[0], T_EFF_int[-1], len(T_EFF_int)*1000)
            y_new = f(x_new)
            T = find_closest(T, x_new)
            indice = np.where(x_new == T)[0]
            y[i][j] = y_new[indice]
            if y[i][j]<0:
                y[i][j] = 0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'atm{T:.0f}_{logg}.int'
    write_file( nome_out, header1, header2, linea, tail)
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return P
def interpolaM(T, logg, M, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    Mi=M
    t_pradk = find_closest(T, T_EFF_Nint)
    T=find_closest(T, T_EFF_Nint) #temperatura più fine (25K) rispetto a logg e M qua sotto (che ancora sono su griglia vanilla)
    logg=find_closest(logg, LOGG_Nint)
    Mint=find_closest(M, MH_int)
    M=find_closest(M, MH_Nint)
    vturbs = vturbo(T, logg, M)
    header1 = f"TEFF   {T}.  GRAVITY {logg:.1f} LTE\nTITLE  [{M:.1f}] VTURB={vturbs}  L/H=1.25 NOVER NEW ODF\n"
    header2 = " OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   0.10000 ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    tail = f"PRADK {getTail(t_pradk, logg, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    for i in range(len(MH_Nint)):
        os.chdir(path_to_atmosphere + f"MH{MH_Nint[i]:.1f}")
        nomeATMint = f'{MH_Nint[i]}T{T:.0f}{logg:.1f}.int'
        copia(f'atmoT{T}{logg:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd())
        P.append(Table.read(nomeATMint, format='ascii'))
    y = np.zeros((72, 10))
    o=-1
    for i in range(72):
        for j in range(10):
            A=[]
            if (find_closest(M, MH_Nint) == 0.5):
                A.append((P[0])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[1])[i][j])
                A.append((P[2])[i][j])
                z = np.polyfit([MH_Nint[0],MH_Nint[1]], np.asarray(A), 2)         
            elif (find_closest(M, MH_Nint) == -2.5):
                A.append((P[-1])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[-2])[i][j])
                A.append((P[-3])[i][j])
                z = np.polyfit([MH_Nint[-1],MH_Nint[-2]], np.asarray(A), 2)  
            elif (find_closest(M, MH_Nint) > -2.5) and (find_closest(M, MH_Nint) < 0.5):
                for k in range(len(MH_Nint)): #scorro su atmosfere
                    if (Mint <= MH_Nint[k-1]) and (Mint > MH_Nint[k]):
                        A.append((P[k-1])[i][j])
                        A.append((P[k])[i][j])
                        A.append((P[k+1])[i][j])
                        z = np.polyfit([MH_Nint[k-1],MH_Nint[k],MH_Nint[k+1]], np.asarray(A), 2)
                        m=(A[1]-A[0])/(MH_Nint[k]-MH_Nint[k-1])
                        o=k-1
            f = np.poly1d(z)
            
            x_new = np.linspace(MH_int[0], MH_int[-1], len(MH_int)*1000)
            y_new = f(x_new)
            Mint = find_closest(Mint, x_new)
            indice = np.where(x_new == Mint)[0]
            #y_new = A[0]+(T-T_EFF_Nint[o])*m
            y[i][j] = y_new[indice]
            if y[i][j]<0:
                y[i][j] = 0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'{Mi:.2f}atm{T:.0f}_{logg}.int'
    write_file( nome_out, header1, header2, linea, tail)
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return P
def Sinterpolalog(T, logg, M, variazioneTemperatura, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    T_in=T
    T=find_closest(T, T_EFF_Nint)              #temperatura su griglia originale
    logg_pradk = find_closest(logg, LOGG_Nint) #per formattazione
    logg=find_closest(logg, LOGG_int)          #logg su griglia interpolata
    M=find_closest(M, MH_Nint)                 #M su griglia originale
    vturbs = vturbo(T, logg, M)                #calcola vturb
    Fe_H = creaHeader_metal(M)
    header1 = f"TEFF   {T_in:.0f}.  GRAVITY {logg:.1f} LTE\nTITLE  [{M:.1f}] VTURB={vturbs:.2f}  L/H=1.25 NOVER NEW ODF\n"
    header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {Fe_H:.3f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    tail = f"PRADK {getTail(T, logg_pradk, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    os.chdir(path_to_atmosphere + f"MH{M:.1f}") #vai nella cartella con le atmosfere della giusta metallicità
    for i in range(len(LOGG_Nint)):             #scorri tutte le atmosfere che hanno temperatura fissata sulla griglia cambiando il logg
        nomeATMint = f'T{T:.0f}{LOGG_Nint[i]:.1f}.int'
        copia(f'atmoT{T}{LOGG_Nint[i]:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd()) #prende solo la tabella 72x10
        P.append(Table.read(f'T{T:.0f}{LOGG_Nint[i]:.1f}.int', format='ascii')) #la metto in una lista, la i-esima atmosfera corrisponde a quella avente T=T_fissato_su_griglia con log(g_i)
    y = np.zeros((72, 10)) 
    for i in range(72): #scorro su righe
        for j in range(10): #scorro su colonne
            a=[]
            for k in range(len(LOGG_Nint)): #scorro su atmosfere
                a.append((P[k])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
            z = np.polyfit(LOGG_Nint, np.asarray(a), 6) #fit polinomiale di 6° grado (molto buono per metallicità circa solare, meno per bassa metallicità)
            f = np.poly1d(z) #f è un polinomio di 6° grado con coefficienti dati da z
            x_new = np.linspace(LOGG_Nint[0], LOGG_Nint[-1], len(LOGG_Nint)*1000) #spanno lo spazio del dominio (1000 volte più grande dell'originale)
            y_new = f(x_new) #creo l'immagine
            LoG = find_closest(logg, x_new) #prendo il logg come quello più vicino a uno dei punti x_new
            indice = np.where(x_new == LoG)[0] #l'indice di x_new_i corretto corrisponde a quello di y_i corretto
            if y[i][j]<0: #non deve mai succedere
                y[i][j]=0
            y[i][j] = y_new[indice] + variazioneTemperatura[i][j]
            if i>0:
                if y[i][j]<y[i-1][j]: #scendendo con le righe deve sempre crescere
                    y[i][j] = y[i-1][j] #se non cresce blocco. In realtà da provare con fit esponenziale oppure facendo media del j-esimo elemento di differenti i-esime righe
                    if (y[i-1][j]==0) and (y[i+1][j]) ==0:
                        y[i][j]=0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'atmL{T_in:.0f}_{logg:.2f}.int'
    write_file( nome_out, header1, header2, linea, tail)
    copia( nome_out, os.getcwd(), path_to_output, nome_out + "aiai")
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return nome_out
def interpolalog(T, logg, M, variazioneTemperatura, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    T_in=T
    T=find_closest(T, T_EFF_Nint)              #temperatura su griglia originale
    logg_pradk = find_closest(logg, LOGG_Nint) #per formattazione
    logg=find_closest(logg, LOGG_int)          #logg su griglia interpolata
    Mint=find_closest(M, MH_int)
    M=find_closest(M, MH_Nint)                 #M su griglia originale
    vturbs = vturbo(T, logg, M)                #calcola vturb
    Fe_H = creaHeader_metal(Mint)
    header1 = f"TEFF   {T_in:.0f}.  GRAVITY {logg:.1f} LTE\nTITLE  [{Mint:.1f}] VTURB={vturbs:.2f}  L/H=1.25 NOVER NEW ODF\n"
    header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {Fe_H:.3f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    tail = f"PRADK {getTail(T, logg_pradk, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    os.chdir(path_to_atmosphere + f"MH{M:.1f}") #vai nella cartella con le atmosfere della giusta metallicità
    for i in range(len(LOGG_Nint)):             #scorri tutte le atmosfere che hanno temperatura fissata sulla griglia cambiando il logg
        nomeATMint = f'T{T:.0f}{LOGG_Nint[i]:.1f}.int'
        copia(f'atmoT{T}{LOGG_Nint[i]:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd()) #prende solo la tabella 72x10
        P.append(Table.read(f'T{T:.0f}{LOGG_Nint[i]:.1f}.int', format='ascii')) #la metto in una lista, la i-esima atmosfera corrisponde a quella avente T=T_fissato_su_griglia con log(g_i)
    y = np.zeros((72, 10)) 
    o=-1
    for i in range(72): #scorro su righe
        for j in range(10): #scorro su colonne
            A=[]
            if (find_closest(logg_pradk, LOGG_Nint) == 0.0):
                A.append((P[0])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[1])[i][j])
                A.append((P[2])[i][j])
                z = np.polyfit([LOGG_Nint[0],LOGG_Nint[1],LOGG_Nint[2]], np.asarray(A), 2)         
            elif (find_closest(logg_pradk, LOGG_Nint) == 5.0):
                A.append((P[-1])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[-2])[i][j])
                A.append((P[-3])[i][j])
                z = np.polyfit([LOGG_Nint[-1],LOGG_Nint[-2],LOGG_Nint[-3]], np.asarray(A), 2)  
            elif (find_closest(logg_pradk, LOGG_Nint) > 0.0) and (find_closest(logg_pradk, LOGG_Nint) < 5.0):
                for k in range(len(LOGG_Nint)): #scorro su atmosfere
                    if (logg >= LOGG_Nint[k-1]) and (logg <= LOGG_Nint[k]):
                        A.append((P[k-1])[i][j])
                        A.append((P[k])[i][j])
                        A.append((P[k+1])[i][j])
                        z = np.polyfit([LOGG_Nint[k-1],LOGG_Nint[k],LOGG_Nint[k+1]], np.asarray(A), 2) #fit polinomiale di 6° grado (molto buono per metallicità circa solare, meno per bassa metallicità)
            f = np.poly1d(z) #f è un polinomio di 6° grado con coefficienti dati da z
            # Y = np.asarray(A)
            # if find_closest(, LOGG_Nint)
            x_new = np.linspace(LOGG_Nint[0], LOGG_Nint[-1], len(LOGG_Nint)*1000) #spanno lo spazio del dominio (1000 volte più grande dell'originale)
            y_new = f(x_new) #creo l'immagine
            # params, cov = curve_fit(expo_func, X, Y, maxfev=1000)
            # a, b, c = params
            # y_new = expo_func(x_new, a, b, c)
            LoG = find_closest(logg, x_new) #prendo il logg come quello più vicino a uno dei punti x_new
            indice = np.where(x_new == LoG)[0] #l'indice di x_new_i corretto corrisponde a quello di y_i corretto
            if y[i][j]<0: #non deve mai succedere
                y[i][j]=0
            y[i][j] = y_new[indice] + variazioneTemperatura[i][j]
            if i>0:
                if y[i][j]<y[i-1][j]: #scendendo con le righe deve sempre crescere
                    y[i][j] = y[i-1][j] #se non cresce blocco. In realtà da provare con fit esponenziale oppure facendo media del j-esimo elemento di differenti i-esime righe
                    if (y[i-1][j]==0) and (y[i+1][j]) ==0:
                        y[i][j]=0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'atmL{T_in:.0f}_{logg:.2f}.int'
    write_file( nome_out, header1, header2, linea, tail)
    copia( nome_out, os.getcwd(), path_to_output, nome_out + "aiai")
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return nome_out
def differenza_INT_grid(temperats, loggheg, metallus, linea=[]):
    T_close = find_closest(temperats, T_EFF_Nint)
    logg_close=find_closest(loggheg, LOGG_Nint)
    M_close=find_closest(metallus, MH_Nint)
    rimuoviHeader_atm(f'atmoT{T_close}{logg_close:.1f}.txt', path_to_atmosphere+f"MH{M_close}", f'atmoT{T_close}{logg_close:.1f}.txt', path_to_output)
    os.chdir(path_to_output)
    tavola = Table.read(f'atmoT{int(T_close)}{logg_close:.1f}.txt', format='ascii')
    rimuoviHeader_atm(f'atm{find_closest(temperats, T_EFF_int)}_{logg_close:.1f}.int', path_to_output, f'atm{temperats:.0f}_{logg_close:.1f}.int', path_to_output)
    tavola1 = Table.read(f'atm{temperats:.0f}_{logg_close:.1f}.int', format='ascii')
    matrice_differenze = np.zeros((72, 10))
    for i in range(72):
        m0=tavola1[i][0]-tavola[i][0]
        m1=tavola1[i][1]-tavola[i][1]
        m2=tavola1[i][2]-tavola[i][2]
        m3=tavola1[i][3]-tavola[i][3]
        m4=tavola1[i][4]-tavola[i][4]
        m5=tavola1[i][5]-tavola[i][5]
        m6=tavola1[i][6]-tavola[i][6]
        m7=tavola1[i][7]-tavola[i][7]
        m8=tavola1[i][8]-tavola[i][8]
        m9=tavola1[i][9]-tavola[i][9]
        matrice_differenze[i][0]=m0
        matrice_differenze[i][1]=m1
        matrice_differenze[i][2]=m2
        matrice_differenze[i][3]=m3
        matrice_differenze[i][4]=m4
        matrice_differenze[i][5]=m5
        matrice_differenze[i][6]=m6
        matrice_differenze[i][7]=m7
        matrice_differenze[i][8]=m8
        matrice_differenze[i][9]=m9
        linea.append(f" {m0:.8E}   {m1:.1f} {m2:.3E} {m3:.3E} {m4:.3E} {m5:.3E} {m6:.3E} {m7:.3E} {m8:.3E} {m9:.3E}\n")
    write_file( "differenza", '', '', linea, '')
    return matrice_differenze
def differenza_INT_grid_M(temperats, loggheg, metallus, linea=[]):
    T_close = find_closest(temperats, T_EFF_Nint)
    logg_close=find_closest(loggheg, LOGG_Nint)
    M_close=find_closest(metallus, MH_Nint)
    rimuoviHeader_atm(f'atmoT{T_close}{logg_close:.1f}.txt', path_to_atmosphere+f"MH{M_close}", f'atmoT{T_close}{logg_close:.1f}.txt', path_to_output)
    os.chdir(path_to_output)
    tavola = Table.read(f'atmoT{int(T_close)}{logg_close:.1f}.txt', format='ascii')
    rimuoviHeader_atm(f'{metallus}atm{find_closest(temperats, T_EFF_Nint)}_{logg_close:.1f}.int', path_to_output, f'{metallus}atm{find_closest(temperats, T_EFF_Nint):.0f}_{logg_close:.1f}.int', path_to_output)
    tavola1 = Table.read(f'{metallus}atm{find_closest(temperats, T_EFF_Nint):.0f}_{logg_close:.1f}.int', format='ascii')
    matrice_differenze = np.zeros((72, 10))
    for i in range(72):
        m0=tavola1[i][0]-tavola[i][0]
        m1=tavola1[i][1]-tavola[i][1]
        m2=tavola1[i][2]-tavola[i][2]
        m3=tavola1[i][3]-tavola[i][3]
        m4=tavola1[i][4]-tavola[i][4]
        m5=tavola1[i][5]-tavola[i][5]
        m6=tavola1[i][6]-tavola[i][6]
        m7=tavola1[i][7]-tavola[i][7]
        m8=tavola1[i][8]-tavola[i][8]
        m9=tavola1[i][9]-tavola[i][9]
        matrice_differenze[i][0]=m0
        matrice_differenze[i][1]=m1
        matrice_differenze[i][2]=m2
        matrice_differenze[i][3]=m3
        matrice_differenze[i][4]=m4
        matrice_differenze[i][5]=m5
        matrice_differenze[i][6]=m6
        matrice_differenze[i][7]=m7
        matrice_differenze[i][8]=m8
        matrice_differenze[i][9]=m9
        linea.append(f" {m0:.8E}   {m1:.1f} {m2:.3E} {m3:.3E} {m4:.3E} {m5:.3E} {m6:.3E} {m7:.3E} {m8:.3E} {m9:.3E}\n")
    write_file( "differenza", '', '', linea, '')
    return matrice_differenze
def creaHeader_metal(Fe_H, metallic=metallic_header, ABUNDANCE_SCALE=ABUNDANCE_SCALE_header):
    return 10**Fe_H
def preparaAbundance(stellaFITS, path_to_abundance=path_to_abundance, eqw_low=5, eqw_high=100, lineList='molte_linee'):
    # Apre il file in modalità lettura
    """Nome del file FITS (doppler corretto) contenente la stella da analizzare
    eqw low and high sono gli estremi di equivalent width accettata nel processo di analisi, per trattare la curva di crescita delle eqw e rimanere fuori del rumore e dalla saturazione della riga"""
    i=0
    os.chdir(path_to_linemake)
              #Iron_lines#
    with open('molte_linee', 'r') as file:
        LINEM = repr(file.read())
    os.chdir(path_to_ARES)
    creaOptFileARES(stellaFITS, lineList)
    ! ./ARES
    with open('stella.ares', 'r') as file:
        ARES = repr(file.read())

    vect1 = LINEM.split("\\n")
    vect2 = ARES.split("\\n")
    VECT1 = np.zeros((len(vect1),9))
    VECT2 = np.zeros((len(vect2),9))

    for i in range(1,len(vect2)-1):
        temp2 = vect2[i].split()
        VECT2[i-1][0]= temp2[0]
        VECT2[i-1][1]= temp2[1]
        VECT2[i-1][2]= temp2[2]
        VECT2[i-1][3]= temp2[3]
        VECT2[i-1][4]= temp2[4]
        VECT2[i-1][5]= temp2[5]
        VECT2[i-1][6]= temp2[6]
        VECT2[i-1][7]= temp2[7]
        VECT2[i-1][8]= temp2[8]
    t4 = Table.read('stella.ares', format='ascii')

    for i in range(1, len(vect1)-1):
        temp1 = vect1[i].split()
        VECT1[i-1][0]= float(temp1[0])
        VECT1[i-1][1]= temp1[1]
        VECT1[i-1][2]= float(temp1[2])
        VECT1[i-1][3]= temp1[3]
        VECT1[i-1][4]= temp1[4]
        VECT1[i-1][5]= 1.0
        VECT1[i-1][6]= 99
        VECT1[i-1][7]= 0

    for i in range(len(VECT1)):
        for j in range(len(VECT2)):
            if (VECT1[i][0] == VECT2[j][0]) and (VECT2[j][4]>eqw_low) and (VECT2[j][4]<eqw_high):
                VECT1[i][7] = VECT2[j][4]

    # for i in range(1, len(vect1)-1): # per ARES linemake
    #     temp1 = vect1[i].split()
    #     VECT1[i-1][0]= float(temp1[0])
    #     VECT1[i-1][1]= temp1[1]
    #     VECT1[i-1][2]= float(temp1[2]) * 8066.01
    #     VECT1[i-1][3]= 0
    #     VECT1[i-1][4]= temp1[3]
    #     VECT1[i-1][5]= 1.0
    #     VECT1[i-1][6]= 99
    #     VECT1[i-1][7]= 0

    ############################## PREPARA FILE PER ABUNDANCE ################################

    os.chdir(path_to_abundance)
    bug = []
    o=0
    T_atmosold=0
    for i in range(len(VECT1)):
        if VECT1[i][7]>eqw_low and VECT1[i][7]<eqw_high: #if VECT1[i][1] == 26.0 and VECT1[i][7]>30:
            bug.append(VECT1[i][7])
            o=o+1
    # indici_righe_da_rimuovere = np.where((VECT1[:, 7] < eqw_low) | (VECT1[:, 7] > eqw_high))[0]
    # VECT1 = np.delete(VECT1, indici_righe_da_rimuovere, axis=0)
    linesout = 'stellantess'
    iron_linesout = 'stellaFes'
    with open(linesout, 'w') as file:
        for i in range(len(VECT1)):
            if VECT1[i][7]!=0:
                file.write(str(VECT1[i][0]))
                file.write('  ')
                file.write(str(VECT1[i][1]))
                file.write('  ')
                file.write(str(VECT1[i][2]))
                file.write('  ')
                file.write(str(VECT1[i][3]))
                file.write('  ')
                file.write(str(VECT1[i][4]))
                file.write('  ')
                file.write(str(VECT1[i][5]))
                file.write('  ')
                file.write(str(VECT1[i][6]))
                file.write('  ')
                file.write(str(VECT1[i][7]))
                file.write('  ')
                file.write('a')
                file.write('\n')

    # with open(iron_linesout, 'w') as file:
    #     for i in range(len(VECT1)):
    #         if VECT1[i][7]!=0 and (VECT1[i][1]==26 or VECT1[i][1]==26.1):
    #             file.write(str(VECT1[i][0]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][1]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][2]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][3]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][4]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][5]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][6]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][7]))
    #             file.write('  ')
    #             file.write('a')
    #             file.write('\n')
    with open(iron_linesout, 'w') as file:
        for i in range(len(VECT1)):
            if VECT1[i][7]!=0:
                file.write(str(VECT1[i][0]))
                file.write('  ')
                file.write(str(VECT1[i][1]))
                file.write('  ')
                file.write(str(VECT1[i][2]))
                file.write('  ')
                file.write(str(VECT1[i][3]))
                file.write('  ')
                file.write(str(VECT1[i][4]))
                file.write('  ')
                file.write(str(VECT1[i][5]))
                file.write('  ')
                file.write(str(VECT1[i][6]))
                file.write('  ')
                file.write(str(VECT1[i][7]))
                file.write('  ')
                file.write('a')
                file.write('\n')
    o=0
    vturb = 1.8
    comando_bash = f"cp {iron_linesout} {path_to_output}\{iron_linesout}"
    os.system(comando_bash)
    os.chdir(path_to_abundance)
def creaInterpolazione(temperat, loggh, metallu):
    interpolaT(temperat, loggh, metallu)
    T_close=find_closest(temperat, T_EFF_Nint)
    logg_close=find_closest(loggh, LOGG_Nint)
    M_close=find_closest(metallu, MH_Nint)
    differenze = differenza_INT_grid(temperat, loggh, metallu)
    interpolaM(temperat, loggh, metallu)
    differenze = differenze + differenza_INT_grid_M(temperat, loggh, metallu)
    b = interpolalog(temperat, loggh, metallu, differenze)
    return b
def abundance_interpolato(temperat, loggh, metallu, nome_stella, path_to_stella=path_to_output):
    olddir=os.getcwd()
    os.chdir(path_to_stella)
    preparaAbundance(nome_stella)
    B=creaInterpolazione(temperat, loggh, metallu)
    sposta(B, path_to_stella, path_to_abundance, "ModelE")
    os.chdir(path_to_abundance)
    creaParamfileAbnd(vturbo(temperat, loggh, metallu))
    SlanciaAbundance()
    os.chdir(path_to_stella)
    bashComm= "mkdir " + f"{nome_stella}"
    os.system(bashComm)
    path_to_output1=path_to_stella+f"{nome_stella}"
    nome_out = f"INT_{metallu}_{temperat}_{loggh}"
    sposta("PINOLOS", path_to_abundance, path_to_output1, nome_out)
    os.chdir(olddir)
    return nome_out
def avvicina_interpol(path_to_stella, nome_file, TEFF, G, M, nome_stella, nome_cartella=path_to_output):
    temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
    temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    sgn=1
    g=-1
    teff=-1
    met=1
    returner = []
    if abs(temp0[1]) > 0.001:
        if (abs(temp0[1]) < 0.06) and (abs(temp0[1]) >= 0.035):
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 70*sgn
        elif abs(temp0[1] > 0.01) and abs(temp0[1]) < 0.035:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 30*sgn
        elif abs(temp0[1]) < 0.01:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 10*sgn
        elif abs(temp0[1]) < 0.13 and abs(temp0[1]) > 0.06:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 150*sgn
        else:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 300*sgn
        crea_ATMinterpol(teff, G, temp0[0], nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{temp2:.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", path_to_output, path_to_output, f"{nome_file1}")
    else:
        nome_file1 = nome_file
        teff = TEFF
    temp0 = simpleAnalysisSingle(f"{path_to_output}", nome_file1, 26.0)
    temp1 = simpleAnalysisSingle(f"{path_to_output}", nome_file1, 26.1)
    sgn1=1
    if (temp0[0]-temp1[0])>0:
        sgn1=-1
    if abs(temp0[0]-temp1[0]) > 0.01:
        if abs(temp0[0]-temp1[0]) < 0.4:
            g = G+(temp0[0]-temp1[0])
        else:
            g = G+((temp0[0]-temp1[0]))
        crea_ATMinterpol(teff, g, temp0[0], nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{g:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{temp2:.2f}_{teff:.0f}_{g:.2f}"
        sposta(f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{g:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        #copia(f"{nome_file1}", path_to_output, path_to_output, f"{nome_file1}{nome_stella}")
        met=temp2
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    return returner
def avvicina_interpol2(path_to_stella, nome_file, temperatu, G, M, nome_stella, nome_cartella=path_to_output):
    temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
    temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    sgn=1
    g=-1.0
    teff=-1.0
    met=1.0
    DG = 0
    DT = 0
    if M-temp0[0] >= 0:
        dM = -(M - temp0[0])/3
    else:
        dM = (M - temp0[0])/3
    returner = []
    dsdg = -0.033574
    dsdt = -0.00023335
    dfdg = -0.5227
    dfdt = 0.001455
    f=-1*((temp0[0])-(temp1[0]))
    s = -1*temp0[1]
    if abs(temp0[1]) > 0.0002 or abs((temp0[0])-(temp1[0])) > 0.0005 or (M - temp0[0])>0.01:
        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
        DG = ((dsdt * f - dfdt * s)/(dfdg * dsdt - dsdg * dfdt))
        DT = ((dfdg * s - dsdg * f)/(dfdg * dsdt - dsdg * dfdt))
        if DG > 0.5:
            DG=0.5*(DG/abs(DG))
        if DT > 250:
            DT = 250*(DT/abs(DT))
        temperatu=temperatu+DT
        teff= temperatu/0.87
        G=G+DG
        crea_ATMinterpol(teff, G, temp0[0], nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{temp0[0]:.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        g=G
        met = temp0[0]
    else:
        nome_file1 = nome_file
        teff = temperatu
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    return returner
def avvicina_interpol3(path_to_stella, nome_file, temperatu, G, M, nome_stella, nome_cartella=path_to_output):
    temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
    temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    sgn=1
    g=-1.0
    teff=-1.0
    met=1.0
    DG = 0
    DT = 0
    returner = []
    dsdg = -0.033574
    dsdt = -0.00023335
    dsdm = 0.0104
    dfdg = -0.5227
    dfdt = 0.001455
    dfdm = -0.3574 
    dydg = -0.13522
    dydt = 0.00041
    dydm = -0.8345
    A = np.array([[dsdg, dsdt, dsdm],
         [dfdg, dfdt, dfdm],
         [dydg, dydt, dydm]])
    A_inv = np.linalg.inv(A)
    nlte_corrF=0#.2
    nlte_corrS=0#-0.027
    nlte_corrM=0#.1
    f=-1*((temp0[0])-(temp1[0]+nlte_corrF))
    s = -1*(temp0[1]+nlte_corrS)
    m = -1*(temp0[0]-(M+nlte_corrM))
    B = np.array([s, f, m])
    if abs(temp0[1]+nlte_corrS) > 0.0001 or abs((temp0[0])-(temp1[0]+nlte_corrF)) > 0.0003 or abs(M - (temp0[0])+nlte_corrM)>0.001:
        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
        DG = (np.dot(A_inv, B))[0]
        DT = (np.dot(A_inv, B))[1]
        DM = (np.dot(A_inv, B))[2]
        if abs(DG) > 0.5:
            DG=0.5*(DG/abs(DG))
        if abs(DT) > 250:
            DT = 250*(DT/abs(DT))
        if abs(DM) > 0.5:
            DM = 0.5*(DM/abs(DM))
        temperatu=temperatu+DT
        G=G+DG
        teff= temperatu
        #temp0[0] = temp0[0]+DM
        crea_ATMinterpol(teff, G, temp0[0]+nlte_corrM, nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{(temp0[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{(temp0[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{(temp0[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        g=G
        met = temp0[0]
    else:
        nome_file1 = nome_file
        teff = temperatu
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    return returner
def leggi_log(path_cartella_log):
    '''Di seguito i nomi delle colonne di log.txt. Restituisce la tavola letta con Table.read(). T      G      M      FeI            FeII           FeI-M/H        FeI-FeII       s(FeI) '''
    olddir = os.getcwd()
    os.chdir(path_cartella_log)
    log_table = Table.read("log.txt", format='ascii')
    return log_table
def minimo_tavola(tab, nome_colonna, toll=2.0, mode='abs'):
    '''Passata una tabella restituisce l'elemento minimo della colonna specificata e il relativo indice di riga. mode=abs trova il minimo in valore assoluto, mode=n trova il minimo considerando anche il segno'''
    x=tab[nome_colonna]
    if mode=='abs':
        X = abs(x)
    if mode=='n':
        X = x
    returner = []
    min = 0
    i=0
    absmin = X[X.argmin()]
    while min < absmin*toll or min-absmin<=0.05:
        M = X.argmin()
        min = X[M]
        min_index = M
        returner.append([min, int(min_index)])
        X[np.where(X == min)] = 1
        i=i+1
    return returner
def final_atmo(path_cartella_log): #
    '''Passa log file, restituisce la best atmosfera. Nelle cartelle contenenti le atmosfere analizzate è possibile creare un log file attraverso la funzione folderLog. Passa qua la directory del log di quella stella. Si cerca prima l'atmosfera che più minimizza FeI-FeII. Da qua si verifica che l'atmosfera che minimizza la slope sia sufficientemente vicina al minimo (i.e. entro 1.2*minimo_slope).'''
    tab=leggi_log(path_cartella_log)
    dF=minimo_tavola(tab, 'FeI-FeII')
    slope=minimo_tavola(tab, 's(FeI)')
    if (tab[slope[0][1]] == tab[dF[0][1]]) and (tab[dF[0][1]]['FeI-M/H'] <= 0.01):
        print(tab[dF[0][1]])
        print()
        return tab[dF[0][1]]
    elif esiste_in_lista(dF[0][1], slope):
        print(tab[dF[0][1]])
        print()
        return tab[dF[0][1]]
    elif esiste_in_lista(slope[0][1], dF):
        print(tab[slope[0][1]])
        print()
        return tab[slope[0][1]]
    elif esiste_in_lista(dF[1][1], slope): 
        print(tab[dF[1][1]])
        print()
        return tab[dF[1][1]]
    elif esiste_in_lista(slope[1][1], dF): 
        print(tab[slope[1][1]])
        print()
        return tab[slope[1][1]]
    elif len(dF)>= 3 and esiste_in_lista(dF[2][1], slope): 
        print(tab[dF[2][1]])
        print()
        return tab[dF[2][1]]
    elif len(slope)>= 3:
        if esiste_in_lista(slope[2][1], dF): 
            print(tab[slope[2][1]])
            print()
            return tab[slope[2][1]]
    else:
        print("nessuna atmosfera trovata.. :C")
def esiste_in_lista(x, lista2):
    esiste = any(x == sub2[1] for sub2 in lista2)
    return esiste
def NLTE_Fe_correction(t, e, g, x, wi):
    url = 'http://www.inspect-stars.com/cp/application.py/A_from_e?element_name=Fe'

    # Cambia il valore del campo 't'
    form_data = {
        't': f'{t}', 
        'e': f'{e}',
        'g': f'{g}',
        'x': f'{x}',
        'wi': f'{wi}'
    }

    # Codifica i dati come query string per l'invio con POST
    encoded_data = urllib.parse.urlencode(form_data).encode('utf-8')

    # Fai la richiesta POST con i dati del form
    req = urllib.request.Request(url, data=encoded_data)

    # Aggiungi un User-Agent per simulare un browser reale
    req.add_header('User-Agent', 'chrome')

    with urllib.request.urlopen(req) as response:
        result = response.read().decode('utf-8')

    # 1. Estrazione del blocco <pre>
    pre_block = re.search(r'<pre>(.*?)</pre>', result, re.DOTALL)
    if pre_block:
        pre_content = pre_block.group(1)  # Ottieni il contenuto tra <pre> e </pre>
        # 2. Estrazione dei numeri (inclusi decimali)
        numbers = re.findall(r'\d+\.\d+|\d+', pre_content)
        # Stampa o utilizza i numeri come preferisci
        print("Numeri estratti:", numbers)
        if len(numbers) == 5:
            return numbers
        else:
            returner = [e, 0, 0, 0, 0]
            return returner
    else:
        print("Blocco <pre> non trovato.")
def find_closest2(num, arr):
    arr = np.array(arr)
    closest_index = np.abs(arr - num).argmin()
    if (arr[closest_index]-num)>0.011:
        print("No line found!")
        return 0
    else:
        return arr[closest_index]
def createCorrectionGrid():
    o=0
    OLDDIR = os.getcwd()
    os.chdir(path_to_correctionGrids)
    with open('correction_grid.txt', 'w') as gridder:
        gridder.write("EW               T               G                V                deltaNLTE\n")
        for E in range(20, 200, 10):
            print(f"EW_riga: {E}")
            for i in range(4100, 5500, 50):
                for j in range(10, 26, 1):
                    for k in range(100, 200, 5):
                        #for l in range(120):
                        U=NLTE_Fe_correction(i, E, j/10, k/100, 34)
                        a0 = int(E-20)
                        a1 = int((i-4000)/20)
                        a2 = int((j-10))
                        a3 = int((k-100)/5)
                        gridder.write(f"{U[0]}               {i}               {j/10}                {k/100}                {U[3]}")
                        gridder.write("\n")
                        o=o+1
                        if o == 200:
                            time.sleep(np.random.exponential(10))
    os.chdir(OLDDIR)



[0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5. ]


In [6]:
#### CONTINUOUS SEARCH BLOCK ####
# for i in range(0, len(b)):    
#     os.chdir(path_to_output)
#     Y = [b[i][0], b[i][1], simpleAnalysisSingle(path_to_output, f"HARPN{i}.fits_INT{b[i][2]:.2f}_{b[i][0]:.0f}_{b[i][1]:.2f}")[0], f"HARPN{i}.fits_INT{b[i][2]:.2f}_{b[i][0]:.0f}_{b[i][1]:.2f}"]
#     for j in range(18):
#         os.system(f"mkdir harpn{i}")        
#         Y=avvicina_interpol(path_to_output, Y2[3], Y[0], Y[1], Y[2], f"H{i}.fits")
#         print(Y)
############################################

for i in range(9, 18):    
    crea_ATMinterpol(4200,1.0,-0.5, f"HARPN{i}.fits")
    crea_ATMinterpol(5000,1.8,-1.5, f"HARPN{i}.fits")
    Y = [4200, 1.0, -0.5, f"HARPN{i}.fits_INT-0.50_4200_1.00"]   # Si specificano le condizioni iniziali della ricerca, nel dubbio si lascino i valori presenti. HARPN0, HARPN1, HARPN2 etc.. sono i nomi delle stelle che ho analizzato.
    Y2 = [5000, 1.80, -1.50, f"HARPN{i}.fits_INT-1.50_5000_1.80"]# Un secondo set di condizioni iniziali viene creato, la ricerca giunge a termine se i due convergono ad un unico risultato, in questo modo non solo esploriamo più spazio con meno rischi di rimane nei minimi locali, ma è anche un modo per valutare la bontà della ricerca
    os.chdir(path_to_output)
    os.system(f"mkdir haru{i}")  ### crea cartella dove inserire i risultati
    o = 0
    for j in range(10): ### I seguenti if sono stati inseriti per riportare l'analisi entro dei range credibili per stelle FGK metal-poor, è un modo per correggere eventuali divergenze nell'esplorazione dello spazio
        if Y[1] <= 0.0:
            Y[1] = 2.5
            Y[0] = 4500

        if Y[0] <= 3700:
            Y[1] = 1.2
            Y[0] = 5100

        if Y[1] > 3.2:
            Y[1] = 0.5
            Y[0] = 4100

        if Y2[1] <= 0.0:
             Y2[1] = 2.5
            Y2[0] = 4500

        if Y2[0] <= 3700:
            Y2[1] = 1.2
            Y2[0] = 5100

        if Y2[1] > 3.2:
            Y2[1] = 0.5
            Y2[0] = 4100
        Y=avvicina_interpol3(path_to_output, Y[3], Y[0], Y[1], Y[2], f"HAPRN{i}.fits")
        Y2=avvicina_interpol3(path_to_output, Y2[3], Y2[0], Y2[1], Y2[2], f"HAPRN{i}.fits")
        copia(f"AH_STELLA{Y[2]:.2f}_{Y[0]:.0f}_{Y[1]:.2f}", path_to_output, path_to_output + f"haru{i}", f"AH_STELLA{Y[2]:.2f}_{Y[0]:.0f}_{Y[1]:.2f}")
        copia(f"AH_STELLA{Y2[2]:.2f}_{Y2[0]:.0f}_{Y2[1]:.2f}", path_to_output, path_to_output + f"haru{i}", f"AH_STELLA{Y2[2]:.2f}_{Y2[0]:.0f}_{Y2[1]:.2f}")
        #copia(f"AH_STELLA{Y2[2]:.2f}_{Y2[0]:.0f}_{Y2[1]:.2f}", path_to_output, path_to_output + f"haru{i}", f"AH_STELLA2{Y2[2]:.2f}_{Y2[0]:.0f}_{Y2[1]:.2f}")
    folderLog(path_to_output+f"haru{i}", 2)

for i in range(9, 18): #questi cicli for servono unicamente a ciclare su diverse stelle
    print(i)
    folderLog(path_to_output+f"haru{i}", 2)

os.chdir(path_to_output)
with open("final_results.txt", "w") as out:
    for i in range(1,17):
        y=final_atmo(path_to_output + f"haru{i}")
        out.write(str(np.asarray(y)))
        out.write('\n')

# #f"HARPN{i}.fits{b[i][2]:.2f}_{b[i][0]:.0f}_{b[i][1]:.2f}"

Bu
23
ga
 T    G     M        FeI           FeII     FeI-M/H FeI-FeII s(FeI)
---- ---- ----- ------------- ------------- ------- -------- ------
4686 1.96 -0.61 -0.611+-0.128 -0.611+-0.044 -0.0007     -0.0 0.0001

Bu
14
ga
 T    G     M        FeI           FeII     FeI-M/H FeI-FeII  s(FeI)
---- ---- ----- ------------- ------------- ------- -------- -------
4447 1.52 -0.78 -0.783+-0.113 -0.783+-0.057 -0.0028      0.0 -0.0001

Bu
13
ga
 T    G     M        FeI           FeII     FeI-M/H FeI-FeII s(FeI)
---- ---- ----- ------------- ------------- ------- -------- ------
4312 1.26 -0.74 -0.743+-0.163 -0.743+-0.062 -0.0033     -0.0    0.0

Bu
19
ga
 T    G     M        FeI           FeII     FeI-M/H FeI-FeII s(FeI)
---- ---- ----- ------------- ------------- ------- -------- ------
4064 0.88 -0.84 -0.837+-0.149 -0.837+-0.224  0.0029     -0.0    0.0

Bu
25
ga
 T    G     M       FeI          FeII     FeI-M/H FeI-FeII  s(FeI)
---- ---- ----- ------------ ------------ ------- -------- ------

In [ ]:
###ITERATIVE SEARCH###

path_to_mainoutput = path_to_output
for I in range(1, N_stelle+1):
    nome_stella = f"HARPN{I}.fits"
    cartella_out = f"HARPN{I}"
    oldfold=os.getcwd()
    os.chdir(path_to_mainoutput)
    comandoBash=f"mkdir HARPN{I}"
    os.system(comandoBash)
    os.chdir(oldfold)
    path_to_output = f"/home/starcat/Scrivania/ANALISIATM/ABBONDANZE_OUT/{cartella_out}/"
    preparaAbundance(nome_stella)
    initial_guesses = [4750, 2.0, -0.5] #temp, logg, M/H
    A = iterative_temperatureSearch(initial_guesses[0], initial_guesses[1], initial_guesses[2], 4)
    A=findBestAtmosphereinFolder(path_to_output, "T")
    tempMH = [abs(A[0] - float(x)) for x in metallicity]
    Meta = metallicity[np.argmin(abs(np.asarray(tempMH)))]
    Meta = float(Meta)
    B = iterative_logGSearch(A[3], initial_guesses[1], Meta, 5)
    B = findBestAtmosphereinFolder(path_to_output, "G")
    C = iterative_temperatureSearch(A[3], B[2], Meta, 5)
    C=findBestAtmosphereinFolder(path_to_output, "T")
    tempMH = [abs(C[0] - float(x)) for x in metallicity]
    Meta = metallicity[np.argmin(abs(np.asarray(tempMH)))]
    Meta = float(Meta)
    D = iterative_logGSearch(C[3], B[2], Meta, 4)
    D = findBestAtmosphereinFolder(path_to_output, "G")
    E = iterative_temperatureSearch(C[3], D[2], Meta, 4)
    E = findBestAtmosphereinFolder(path_to_output, "T", 1, 0.2)
    with open(f"HARPN{I}", 'w') as file:
        file.write('FeI: ')
        file.write(str(E[0]))
        file.write('\nSlope:  ')
        file.write(str(E[1]))
        file.write('\nstddev:  ')
        file.write(str(E[2]))
        file.write('\nTemperatura:  ')
        file.write(str(E[3]))
        file.write('\nlog(g):  ')
        file.write(str(E[4]))
        file.write('  ')
        file.write('\nFeII-FeI:  ')
        file.write(str(E[5]-E[0]))
        file.write('  ')
    os.system(f"mkdir harp{I}")
    sposta(f"TSTELLA_{E[0]:.1f}_{E[3]:.0f}_{E[4]:.1f}", os.getcwd(), f"{os.getcwd()}/harp{I}", f"TSTELLA_{E[0]:.1f}_{E[3]:.0f}_{E[4]:.1f}")
    E = findBestAtmosphereinFolder(path_to_output, "T")
    with open(f"HARPN{I}_IIb", 'w') as file:
        file.write('FeI: ')
        file.write(str(E[0]))
        file.write('\nSlope:  ')
        file.write(str(E[1]))
        file.write('\nstddev:  ')
        file.write(str(E[2]))
        file.write('\nTemperatura:  ')
        file.write(str(E[3]))
        file.write('\nlog(g):  ')
        file.write(str(E[4]))
        file.write('  ')
        file.write('\nFeII-FeI:  ')
        file.write(str(E[5]-E[0]))
        file.write('  ')